In [1]:
import os
str1 = '$'
file_list = []
os.chdir(r"C:\Users\ThinkPad\Desktop\银行交易流水Path")
for path_name, dir, files_name in os.walk(os.getcwd()):
    for file in files_name:
        if str1 not in file:
            file_list.append(os.path.join(path_name, file))
            
print(len(file_list))
print('\n'.join(file_list))



import pandas as pd
df_group = pd.DataFrame(columns = ['Path','Columns','False','True'])
type1 = '.xls'
type2 = '.csv'
type3 = '.et'
output_path = 'C:\\Users\\ThinkPad\\Desktop\\银行交易流水output\\'
for i in range(len(file_list)):
    if type1 in file_list[i]:
        df = pd.read_excel(file_list[i], encoding = "gbk")
        df = df.loc[:,~df.columns.str.contains('^Unnamed')]
    elif type2 in file_list[i]:
        df = pd.read_csv(file_list[i], encoding = 'gbk')
        df = df.loc[:,~df.columns.str.contains('^Unnamed')]
    else:
        continue
    print(i ,end = ' ')
    df.columns = df.columns.str.strip()
    col_name = df.columns
    bool_S = df.duplicated().value_counts()
    if len(bool_S)==1:
        df_group.loc[i] = {'Path':file_list[i],'Columns':tuple(col_name),'False':bool_S[0],'True':0}
    elif len(bool_S)==2:
        df_group.loc[i] = {'Path':file_list[i],'Columns':tuple(col_name),'False':bool_S[0],'True':bool_S[1]}
grouped = df_group.groupby('Columns')
groupname_list = list(grouped.size().index)
print('以下'+str(len(groupname_list))+'种格式数据分类，保留流水、账户类别')
print(grouped.size())

1
C:\Users\ThinkPad\Desktop\银行交易流水Path\银行交易流水Path.xlsx
0 以下1种格式数据分类，保留流水、账户类别
Columns
(银行, 客户名称, 客户证件号码, 查询账号, 查询卡号, 交易对方名称, 交易对方证件号码, 交易对方账号, 交易对方卡号, 交易类型, 借贷标志, 币种, 交易金额, 交易余额, 交易时间, 交易流水号, 交易对方余额, 交易对方账号开户行, 交易摘要, 交易网点名称, 交易网点代码, 日志号, 传票号, 凭证种类, 凭证号, 现金标志, 终端号, 交易是否成功, 交易发生地, 商户名称, 商户号, IP地址, MAC地址, 交易柜员号, 备注)    1
dtype: int64


In [5]:
# group_账户 = grouped.get_group(groupname_list[3])
# group_账户['文件序号I'] = group_账户.index

group_流水2 = grouped.get_group(groupname_list[0])
group_流水2['文件序号I'] = group_流水2.index


# with pd.ExcelWriter(output_path + "文件目录.xlsx") as writer:
#     group_流水.to_excel(writer, sheet_name = '流水文件目录',index = None)
#     group_流水2.to_excel(writer, sheet_name ='流水文件目录-标准',index = None)
#     group_账户.to_excel(writer, sheet_name ='账户文件目录',index = None)
# print('文件目录Done!')


import numpy as np
import re
import datetime
df_忽略行 = pd.DataFrame()
df_冲正 = pd.DataFrame()
all_流水 = pd.DataFrame()
all_账户 = pd.DataFrame()
all_证件 = pd.DataFrame()
all_任务 = pd.DataFrame()
for i in range(len(file_list)):
    if file_list[i] in list(group_流水2['Path']) :
        if file_list[i] in list(group_流水2['Path']):  #经侦云流水转换AI模板标准银行交易流水
            if type1 in file_list[i] or type3 in file_list[i]:
                df = pd.read_excel(file_list[i],encoding = "gbk",dtype = "object")
            elif type2 in file_list[i]:
                df = pd.read_csv(file_list[i], engine="python")
            df.columns = df.columns.str.strip()
            df = df.applymap(lambda x: re.sub('nan','',str(x)))
            col_name = df.columns.tolist()
        print(i,end=' ')
        df['path'] = file_list[i]
        df['序号'] = df.index
        df = df.applymap(lambda x: re.sub('\t','',str(x)).strip())  # 去掉每个元素的\t
        df['查询卡号'] = df['查询卡号'].map(lambda x: np.NaN if str(x)=='' or str(x)==' ' else x)
        df['查询卡号'][df['查询卡号'].isnull()]=df['查询账号'][df['查询卡号'].isnull()] # 用账号填充卡号空白
        df_忽略行 = df_忽略行.append(df[(df['交易金额']=='') | (df['交易金额']=='交易金额') | (df['查询卡号']=='')| (df['交易时间']=='')])
        df = df.drop(df[(df['交易金额']=='') | (df['交易金额']=='交易金额') | (df['查询卡号']=='') | (df['交易时间']=='')].index)  # 记录后删除交易金额为空或者账号卡号都为空的无效流水行
        df['交易金额'] = df['交易金额'].map(lambda x: float(x))  #交易金额指定浮点型
        df['交易余额'] = df['交易余额'].map(lambda x: np.NaN if str(x)=='' else x)
        df['交易余额'] = df['交易余额'].map(lambda x: float(x))  #交易余额指定浮点型
        df['借贷标志'] = df['借贷标志'].map(lambda x:'出' if x=='借' else ('进' if x=='贷' else x))
        df = df.drop_duplicates(subset = ['查询卡号','交易时间','交易金额','交易余额','借贷标志','交易对方账号'],keep = 'last')  #去除重复
        df = df.sort_values(by = ['查询卡号','交易时间','交易金额'],ascending=[True,True,False]).reset_index().iloc[:,1:] # 时间升序排序更新index
        print(str(i)+'. 单表去重后记录数:'+str(df.shape[0]),end = ' ')
        df_负 = df[df['交易金额']<0]
        for j in range(0,len(df_负)):
            df_正 = df[(df['交易金额']== abs(df_负['交易金额'].iloc[j])) & \
                (df['借贷标志']== df_负['借贷标志'].iloc[j]) & \
                (df['交易时间']<= df_负['交易时间'].iloc[j]) & \
                (df['查询卡号']== df_负['查询卡号'].iloc[j]) & \
                (
                 (df['交易对方账号']== df_负['交易对方账号'].iloc[j]) | \
                 ((df['交易对方账号'] == '') & (df_负['交易对方账号'].iloc[j] == '')) | \
                 ((df['交易对方账号'] != '') & (df_负['交易对方账号'].iloc[j] == '')) | \
                 ((df['交易对方账号'] == '') & (df_负['交易对方账号'].iloc[j] != '')) | \
                 ((df['交易对方账号'] != '') & (df['查询卡号']==df_负['交易对方账号'].iloc[j]))
                )
                ]
            d_labels = [df_正.index.max(),df_负.index[j]]  # 一对冲正匹配记录的索引
            if len(df_正)>0:
                df_冲正 = df_冲正.append(df.loc[d_labels])
                df = df.drop(d_labels) # 记录后删除一对冲正匹配的无效流水行
                print(d_labels)
            elif len(df_正)==0:
                print(d_labels)
        print('剔除冲正后保留记录数:'+str(df.shape[0]))
        all_流水 = all_流水.append(df)
#     elif file_list[i] in list(group_账户['Path']):
#         if type1 in file_list[i]:
#             df_account = pd.read_excel(file_list[i],encoding = "gbk",dtype = "object")
#         elif type2 in file_list[i]:
#             df_account = pd.read_csv(file_list[i], engine="python")
#         df_account.columns = df_account.columns.str.strip()
#         df_account['path'] = file_list[i]
#         df_account = df_account.applymap(lambda x: re.sub('\t','',str(x)).strip())
#         df_account = df_account.applymap(lambda x: re.sub('nan','',str(x)).strip())
#         all_账户 = all_账户.append(df_account)

all_流水.to_csv(output_path + "流水合并剔除冲正.csv",index = None)
print(all_流水.shape)
with pd.ExcelWriter(output_path + "剔除行.xlsx") as writer:
    df_冲正.to_excel(writer, sheet_name='冲正全',index = None)
    df_忽略行.to_excel(writer, sheet_name='忽略行',index = None)
print(df_冲正.shape,end = ' ')
print(df_忽略行.shape)

# all_账户['查询卡号'] = all_账户['查询卡号'].str.split('_', 1,expand = True).iloc[:,0]   # 账户开户信息中卡号去掉_后缀
# all_账户['查询卡号'] = all_账户['查询卡号'].str.split('-', 1,expand = True).iloc[:,0]  # 账户开户信息中卡号去掉-后缀
# all_账户 = all_账户.drop_duplicates(subset = list(group_账户['Columns'].iloc[0]),keep = 'last')
# all_账户.to_excel(output_path + "账户合并.xlsx",index = None)
# print(all_账户.shape)

print('合并Done!')



# # 卡号修正
# # 汇总时已经填充了查询账号，如果查询卡号、查询账号是唯一对应关系，实际应该依赖查询账号填充对应的查询卡号
# df = all_流水.sort_values(by = ['查询卡号','交易时间','交易金额'],ascending=[True,True,False]).reset_index().iloc[:,1:]  # 流水汇总的all_流水 更新索引

# # 手工微调
# df['查询卡号'] = df['查询卡号'].str.split('_', 1,expand = True).iloc[:,0] 
# df['查询卡号'] = df['查询卡号'].str.split('-', 1,expand = True).iloc[:,0]

# df['交易对方账号']= df['交易对方账号'].replace('21*****90','215500690')
# df['交易对方账号']= df['交易对方账号'].replace('24*****33','243300133')
# df['交易对方账号']= df['交易对方账号'].replace('21*****93','215500693')
# df['交易对方账号']= df['交易对方账号'].replace('22*****12','220800312')

# print('手工微调后',end = '')
# print(df.shape)



# def groupa1(df_g):
#     return df_g.groupby(['查询账号','查询卡号'],as_index = False).size()
# def groupa2(df_g):
#     return df_g.groupby(['查询卡号'],as_index = False).size()


# df_账号 = df[(df['查询卡号']!=df['查询账号']) & (df['查询账号']!='')] #  提取查询卡号、查询账号的对应关系表
# df_a1 = df_账号.groupby(['查询账号','查询卡号']).size().reset_index()
# df_a2 = df_a1.groupby(['查询账号'],as_index = False).size()
# card = df_a1[df_a1['查询账号'].isin(df_a2[df_a2==1].index)]


# df_开户a1 = groupa1( df ).reset_index()
# df_开户a2 = groupa2(df_开户a1)
# card_明细仅账号 = df_开户a1[ ( df_开户a1['查询账号'].isin(df_开户a2[df_开户a2==1].index) )&( df_开户a1['查询账号'] == df_开户a1['查询卡号'] )]
# card_开户对应卡号 = groupa1( all_账户[all_账户['查询账号'].isin( card_明细仅账号['查询账号'] )] ).reset_index()
# card = card.append(card_开户对应卡号)

# card.to_excel(output_path + "卡号账号唯一对应.xlsx",index = None)

# print('账号卡号对应Done!')
# dfs0 = pd.merge(df,card,how = 'left',on='查询账号')
# print(dfs0.columns)
# dfs0['查询卡号_x'][(dfs0['查询卡号_x']!=dfs0['查询卡号_y']) & (dfs0['查询卡号_y'].notnull())]=dfs0['查询卡号_y'][(dfs0['查询卡号_x']!=dfs0['查询卡号_y']) & (dfs0['查询卡号_y'].notnull())]
# all_流水卡修正 = dfs0.iloc[:,:34]
# all_流水卡修正.columns = all_流水.columns
# # all_流水卡修正.to_csv(output_path + "流水卡号.csv",index = None)
# print(all_流水卡修正.shape)


# df_账号 = all_流水卡修正[ (all_流水卡修正['查询账号']!='')] 
# df_a1 = df_账号.groupby(['查询账号','查询卡号']).size().reset_index()
# df_a2 = df_a1.groupby(['查询账号'],as_index = False).size()
# for j in range(2,df_a2.max()+1):
#     for k in range(len(df_a2[df_a2==j])):
#         print(df_a1[df_a1['查询账号']==(df_a2[df_a2==j].index[k])])

#         df_换卡 = all_流水卡修正.groupby('查询账号').get_group(df_a2[df_a2==j].index[k])
#         df_换卡 = df_换卡.sort_values(by = ['查询账号','交易时间','交易金额'],ascending=[True,True,False]).reset_index().iloc[:,1:]
#         df_换卡最后一条 = df_换卡[df_换卡['查询卡号']!=df_换卡['查询账号']].drop_duplicates(subset = ['查询卡号'],keep = 'last')
#         df_换卡第一条 = df_换卡[df_换卡['查询卡号']!=df_换卡['查询账号']].drop_duplicates(subset = ['查询卡号'],keep = 'first')
#         for i in range(1,j):
#             if i == len(df_换卡第一条):
#                 all_流水卡修正['查询卡号']       =       all_流水卡修正['查询卡号'].replace(df_换卡最后一条.iloc[i-1,1],df_换卡第一条.iloc[i-1,0])
#                 all_流水卡修正['交易对方账号'] = all_流水卡修正['交易对方账号'].replace(df_换卡最后一条.iloc[i-1,1],df_换卡第一条.iloc[i-1,0])
#                 print(i, end = ' ')
#                 print('  replace("'+ str(df_换卡最后一条.iloc[i-1,1])+'","'+str(df_换卡第一条.iloc[i-1,0])+'")' )
#             elif (   ((df_换卡第一条.iloc[i,8]=='进') and (abs(df_换卡最后一条.iloc[i-1,7] + df_换卡第一条.iloc[i,6] - df_换卡第一条.iloc[i,7]) < 0.0001) ) 
#                 or ((df_换卡第一条.iloc[i,8]=='出') and (abs(df_换卡最后一条.iloc[i-1,7] - df_换卡第一条.iloc[i,6] - df_换卡第一条.iloc[i,7]) < 0.0001) ) ):
#                 all_流水卡修正['查询卡号']       =       all_流水卡修正['查询卡号'].replace(df_换卡最后一条.iloc[i-1,0],df_换卡第一条.iloc[i,0])
#                 all_流水卡修正['交易对方账号'] = all_流水卡修正['交易对方账号'].replace(df_换卡最后一条.iloc[i-1,0],df_换卡第一条.iloc[i,0])
#                 print(i, end = ' ')
#                 print('replace("'+ str(df_换卡最后一条.iloc[i-1,0])+'","'+str(df_换卡第一条.iloc[i,0])+'")' )

#             elif all(  (     (df_换卡第一条.iloc[i,8]=='进')
#                       & (abs(df_换卡最后一条.iloc[i-1,7] + df_换卡第一条.iloc[i,6] - df_换卡第一条.iloc[i,7]) > 0.0001) 
#                       & (abs(df_换卡.iloc[df_换卡第一条.iloc[[i],:].index-1,7] + df_换卡第一条.iloc[i,6] - df_换卡第一条.iloc[i,7]) < 0.0001) 
#                      ) 
#                  | (     (df_换卡第一条.iloc[i,8]=='出')
#                       & (abs(df_换卡最后一条.iloc[i-1,7] - df_换卡第一条.iloc[i,6] - df_换卡第一条.iloc[i,7]) > 0.0001) 
#                       & (abs(df_换卡.iloc[df_换卡第一条.iloc[[i],:].index-1,7] - df_换卡第一条.iloc[i,6] - df_换卡第一条.iloc[i,7]) < 0.0001) 
#                      )
#                  ):
#                 if all(( (df_换卡.iloc[df_换卡最后一条.iloc[[i-1],:].index+1,8]=='进') &\
#                       (abs(df_换卡最后一条.iloc[i-1,7] + df_换卡.iloc[df_换卡最后一条.iloc[[i-1],:].index+1,6] - df_换卡.iloc[df_换卡最后一条.iloc[[i-1],:].index+1,7]) < 0.0001) 
#                      ) | \
#                     ( (df_换卡.iloc[df_换卡最后一条.iloc[[i-1],:].index+1,8]=='出') & \
#                       (abs(df_换卡最后一条.iloc[i-1,7] - df_换卡.iloc[df_换卡最后一条.iloc[[i-1],:].index+1,6] - df_换卡.iloc[df_换卡最后一条.iloc[[i-1],:].index+1,7]) < 0.0001) 
#                      )
#                    ):
#                     if df_换卡最后一条.iloc[i-1,0] != df_换卡第一条.iloc[i,0]:# 换卡中间穿插卡号为空，但是换卡前&空&换卡后余额有连续衔接的
#                         all_流水卡修正['查询卡号']       =       all_流水卡修正['查询卡号'].replace(df_换卡最后一条.iloc[i-1,0],df_换卡第一条.iloc[i,0])
#                         all_流水卡修正['交易对方账号'] = all_流水卡修正['交易对方账号'].replace(df_换卡最后一条.iloc[i-1,0],df_换卡第一条.iloc[i,0])
#                         print(i, end = ' ')
#                         print('replace("'+ str(df_换卡最后一条.iloc[i-1,0])+'","'+str(df_换卡第一条.iloc[i,0])+'")')
#                     else:                                                   # 不是换卡，穿插在一张卡中间的另一张卡，但是穿插前后余额有连续衔接的 
#                         all_流水卡修正['查询卡号']       =       all_流水卡修正['查询卡号'].replace(df_换卡最后一条.iloc[i-1,0],df_换卡.iloc[df_换卡最后一条.iloc[[i-1],:].index+1,0].iloc[0])
#                         all_流水卡修正['交易对方账号'] = all_流水卡修正['交易对方账号'].replace(df_换卡最后一条.iloc[i-1,0],df_换卡.iloc[df_换卡最后一条.iloc[[i-1],:].index+1,0].iloc[0])
#                         print(i, end = ' ')
#                         print('replace("'+ str(df_换卡最后一条.iloc[i-1,0])+'","'+str(df_换卡.iloc[df_换卡最后一条.iloc[[i-1],:].index+1,0].iloc[0])+'")')
#             else:
#                 print('否')
#                 break



# # # 户名填充
# # # 方法一：依赖账户开户信息表汇总的all_账户
# # df_账户 = all_账户[all_账户['账户开户名称'] != '']

# # #  执行了手动修正卡号则屏蔽
# # df_流水 = all_流水卡修正  #  df_流水 是手动修正卡号后的流水汇总 ，不需要手动就用账号卡号唯一关系修正的 all_流水卡修正 

# # dfs = pd.merge(df_流水,df_账户.drop_duplicates(subset='查询卡号'),how = 'left',on = '查询卡号').iloc[:,:(df_流水.shape[1]+2)]
# # dfs['客户名称'][dfs['账户开户名称'].notnull()] = dfs['账户开户名称'][dfs['账户开户名称'].notnull()]
# # dfs['客户证件号码'][dfs['开户人证件号码'].notnull()] = dfs['开户人证件号码'][dfs['开户人证件号码'].notnull()]
# # df = dfs.iloc[:,:34]
# # df.columns = df_流水.columns
# # print(df.shape)


# # 方法二：依赖交易对手卡号、交易对方名称的唯一对应关系
# df_户名 = df[(df['交易对方账号'].isin(df['查询卡号'][df['客户名称']==''])) & (df['交易对方名称']!='')]  # 对手卡号中的主账号且作为对手卡号出现时交易对方名称非空，作为本方卡号出现时户名缺失
# df_a1 = df_户名.groupby(['交易对方账号','交易对方名称']).size().reset_index()
# df_a2 = df_a1.groupby(['交易对方账号'],as_index=False).size()
# account2 = df_a1[df_a1['交易对方账号'].isin(df_a2[df_a2==1].index)]
# account2.to_excel(output_path + "卡号户名唯一对应.xlsx",index = None)
# print('卡号户名对应Done!')
# dfs2 = pd.merge(df,account2,how = 'left',left_on='查询卡号',right_on='交易对方账号')
# print(dfs2.columns)
# dfs2['客户名称'][dfs2['交易对方名称_y'].notnull()] = dfs2['交易对方名称_y'][dfs2['交易对方名称_y'].notnull()]
# df = dfs2.iloc[:,:34]
# df.columns = df_流水.columns
# print(df.shape)


# df_流水户名 = df
# df_流水户名.to_csv(output_path +"流水户名.csv",index = None)
# print('流水户名Done!')

# # #--------------------------------------------------保留7月以后---------------------------------------------------------#
# # df = df[(df['交易时间'].str[0:4].isin(['2020']) ) & (df['交易时间'].str[5:7].isin(['07','08','09','10','11']) )]
# # #--------------------------------------------------保留7月以后---------------------------------------------------------#

# for k in df['客户名称'].drop_duplicates():
#     if k !='':
#         if not os.path.exists(output_path + k):
#             os.mkdir(output_path + k)
#         print(output_path + k)

# df_S1 = df.groupby('查询卡号').size().reset_index()
# df_S2 = pd.DataFrame(columns = ['查询卡号','Size','False','True'])
# df_S3 = pd.DataFrame(columns = ['查询卡号','客户名称','证件号码','夜间交易次数','最早时间','最晚时间','余额','总和','净和','入向','出向','入向对手数','出向对手数','收付检验起始值','Path'])
# for i in df_S1.index:
#     df_every = df.groupby('查询卡号').get_group(df_S1['查询卡号'][i])
#     bool_S = df_every.duplicated(subset = ['查询卡号','交易时间','交易金额','交易余额','借贷标志','交易对方账号','币种'],keep = 'last').value_counts()
#     df_S2.loc[i] = {'查询卡号':df_every['查询卡号'].iloc[0],
#                         'Size':df_every.shape[0],
#                         'False':bool_S[0],
#                         'True':bool_S.map(lambda x:0 if len(bool_S)==1 else bool_S[1]).values[0]
#                     }
#     df_every = df_every.drop_duplicates(subset = ['查询卡号','交易时间','交易金额','交易余额','借贷标志','交易对方账号','币种'],keep = 'last')
#     if '交易明细信息' in df_every['path'].iloc[0]:
# #         户名+卡号+银行
#         df_every.to_excel(output_path+df_every['客户名称'].iloc[0]+"\\" + str(df_every['客户名称'].iloc[0])+str(df_every['查询卡号'].iloc[0])+str(df_every['path'].str.split('\\\\').iloc[0][-1]).split('银行')[0]+".xlsx",index = None)
#     else:
# #         户名+ 卡号
#         df_every.to_excel(output_path+df_every['客户名称'].iloc[0]+"\\" + str(df_every['客户名称'].iloc[0])+str(df_every['查询卡号'].iloc[0])+".xlsx",index = None)
#     df_night = df_every[df_every['交易时间'].str[11:13].isin(['00','01','02','03','04','05','06'])]
#     df_S3.loc[i] = {'查询卡号':df_every['查询卡号'].iloc[0],
#                     '客户名称':df_every['客户名称'].iloc[0],
#                     '证件号码':df_every['客户证件号码'].iloc[0],
#                     '夜间交易次数':len(df_night),
#                     '最早时间':df_every['交易时间'].min(),
#                     '最晚时间':df_every['交易时间'].max(),
#                     '余额':df_every['交易余额'].loc[df_every.index.max()],
#                     '总和':df_every['交易金额'].sum(),
#                     '净和':round(df_every['交易金额'][df_every['借贷标志']=='进'].sum()-df_every['交易金额'][df_every['借贷标志']=='出'].sum(),
#                     '入向':df_every['交易金额'][df_every['借贷标志']=='进'].sum(),
#                     '出向':df_every['交易金额'][df_every['借贷标志']=='出'].sum(),
#                     '入向对手数':len(df_every['交易对方账号'][df_every['借贷标志']=='进'].drop_duplicates()),
#                     '出向对手数':len(df_every['交易对方账号'][df_every['借贷标志']=='出'].drop_duplicates()),
#                     '收付检验起始值':df_every['交易余额'].iloc[0]+df_every.apply(lambda x: x['交易金额'] if x['借贷标志']=='出' else -x['交易金额'],axis=1).iloc[0],
#                     'Path':'\n'.join(df_every['path'].drop_duplicates().tolist())
#                    }
#     print(i,end = ' ')
# df_S = pd.merge(df_S3,df_S2,how = 'left',on = '查询卡号')

# dfs_账户银行 = pd.merge(df_S,df_账户.drop_duplicates(subset='查询卡号'),how = 'left',on = '查询卡号')
# df_S['开户网点'] = dfs_账户银行['开户网点']
# df_S['账户状态'] = dfs_账户银行['账户状态']
# df_S.to_excel(output_path + "交易统计.xlsx",index = None)
# # 统计完了，按照六大要素+币种 删除重复项（因为手动改了-101与-102的双向交易的卡号后缀，-101出-102进这类记录会造成自实体链接，不影响）
# all_流水完整 = df.drop_duplicates(subset = ['查询卡号','交易时间','交易金额','交易余额','借贷标志','交易对方账号','币种'],keep = 'last')
# all_流水完整.to_csv(output_path + "流水完整.csv",index = None)
# print(all_流水完整.shape)

0 

D:\Users\ThinkPad\anaconda3\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0. 单表去重后记录数:4173 [122, 123]
[886, 887]
[911, 912]
[920, 921]
[935, 936]
[943, 944]
[945, 946]
[947, 948]
[1086, 1087]
[1091, 1092]
[1114, 1115]
[1121, 1122]
[1126, 1127]
[1161, 1162]
[1167, 1168]
[1182, 1183]
[1209, 1210]
[1219, 1220]
[1222, 1223]
[1230, 1231]
[1367, 1368]
[1381, 1382]
[1384, 1385]
[1389, 1390]
[1445, 1446]
[1461, 1462]
[4019, 4030]
[4012, 4031]
[4003, 4032]
[4112, 4113]
剔除冲正后保留记录数:4113
(4113, 37)
(60, 37) (0, 37)
合并Done!


In [86]:
import pandas as pd
import numpy as np
import re
import os
import datetime
output_path = 'O:\\舒城最近清洗\\'

all_流水 = pd.read_csv(output_path + "流水合并剔除冲正.csv",encoding = 'utf-8',dtype = 'object')
all_流水 = all_流水.applymap(lambda x: re.sub('nan','',str(x)))
all_流水['交易金额'] = all_流水['交易金额'].map(lambda x: float(x))  #交易金额指定浮点型
all_流水['交易余额'] = all_流水['交易余额'].map(lambda x: np.NaN if str(x)=='' else x)
all_流水['交易余额'] = all_流水['交易余额'].map(lambda x: float(x))

all_账户 = pd.read_excel(output_path + "账户合并.xlsx",encoding = 'gbk',dtype = 'object')
all_账户 = all_账户.applymap(lambda x: re.sub('nan','',str(x)))

# 卡号修正
# 汇总时已经填充了交易账号，如果交易卡号、交易账号是唯一对应关系，实际应该依赖交易账号填充对应的交易卡号
df = all_流水.sort_values(by = ['交易卡号','交易时间','交易金额'],ascending=[True,True,False]).reset_index().iloc[:,1:]  # 流水汇总的all_流水 更新索引

# 手工微调
df['交易卡号'] = df['交易卡号'].str.split('_', 1,expand = True).iloc[:,0] 
df['交易卡号'] = df['交易卡号'].str.split('-', 1,expand = True).iloc[:,0]

# df = df[~((df['交易对手账卡号']=='6236681460012372435') & (df['对手户名']=='吴晶晶'))]
# df = df[~((df['交易对手账卡号']=='6230780100017400236') & (df['对手户名']=='吴晶晶'))]
# df = df[~((df['交易对手账卡号']=='6228411070343158413') & (df['对手户名']=='毛霓姣'))]
# df = df[~((df['交易对手账卡号']=='6212261207008544728') & (df['对手户名']=='唐文强'))]

# df['对手户名'] = df['对手户名'].replace('邓?婷','邓雲婷')

df['交易对手账卡号']= df['交易对手账卡号'].replace('21*****90','215500690')
df['交易对手账卡号']= df['交易对手账卡号'].replace('24*****33','243300133')
df['交易对手账卡号']= df['交易对手账卡号'].replace('21*****93','215500693')
df['交易对手账卡号']= df['交易对手账卡号'].replace('22*****12','220800312')


# df['交易户名'][df['交易卡号']=='6230361104131659666'] = df['交易户名'][df['交易卡号']=='6230361104131659666'].replace('','范金锋')

print('手工微调后',end = '')
print(df.shape)



def groupa1(df_g):
    return df_g.groupby(['交易账号','交易卡号'],as_index = False).size()
def groupa2(df_g):
    return df_g.groupby(['交易卡号'],as_index = False).size()


df_账号 = df[(df['交易卡号']!=df['交易账号']) & (df['交易账号']!='')] #  提取交易卡号、交易账号的对应关系表
df_a1 = df_账号.groupby(['交易账号','交易卡号']).size().reset_index()
df_a2 = df_a1.groupby(['交易账号'],as_index = False).size()
card = df_a1[df_a1['交易账号'].isin(df_a2[df_a2==1].index)]


df_开户a1 = groupa1( df ).reset_index()
df_开户a2 = groupa2(df_开户a1)
card_明细仅账号 = df_开户a1[ ( df_开户a1['交易账号'].isin(df_开户a2[df_开户a2==1].index) )&( df_开户a1['交易账号'] == df_开户a1['交易卡号'] )]
card_开户对应卡号 = groupa1( all_账户[all_账户['交易账号'].isin( card_明细仅账号['交易账号'] )] ).reset_index()
card = card.append(card_开户对应卡号)

card.to_excel(output_path + "卡号账号唯一对应.xlsx",index = None)

print('账号卡号对应Done!')
dfs0 = pd.merge(df,card,how = 'left',on='交易账号')
print(dfs0.columns)
dfs0['交易卡号_x'][(dfs0['交易卡号_x']!=dfs0['交易卡号_y']) & (dfs0['交易卡号_y'].notnull())]=dfs0['交易卡号_y'][(dfs0['交易卡号_x']!=dfs0['交易卡号_y']) & (dfs0['交易卡号_y'].notnull())]
all_流水卡修正 = dfs0.iloc[:,:34]
all_流水卡修正.columns = all_流水.columns
# all_流水卡修正.to_csv(output_path + "流水卡号.csv",index = None)
print(all_流水卡修正.shape)


df_账号 = all_流水卡修正[ (all_流水卡修正['交易账号']!='')] 
df_a1 = df_账号.groupby(['交易账号','交易卡号']).size().reset_index()
df_a2 = df_a1.groupby(['交易账号'],as_index = False).size()
for j in range(2,df_a2.max()+1):
    for k in range(len(df_a2[df_a2==j])):
        print(df_a1[df_a1['交易账号']==(df_a2[df_a2==j].index[k])])

        df_换卡 = all_流水卡修正.groupby('交易账号').get_group(df_a2[df_a2==j].index[k])
        df_换卡 = df_换卡.sort_values(by = ['交易账号','交易时间','交易金额'],ascending=[True,True,False]).reset_index().iloc[:,1:]
        df_换卡最后一条 = df_换卡[df_换卡['交易卡号']!=df_换卡['交易账号']].drop_duplicates(subset = ['交易卡号'],keep = 'last')
        df_换卡第一条 = df_换卡[df_换卡['交易卡号']!=df_换卡['交易账号']].drop_duplicates(subset = ['交易卡号'],keep = 'first')
        for i in range(1,j):
            if i == len(df_换卡第一条):
                all_流水卡修正['交易卡号']       =       all_流水卡修正['交易卡号'].replace(df_换卡最后一条.iloc[i-1,1],df_换卡第一条.iloc[i-1,0])
                all_流水卡修正['交易对手账卡号'] = all_流水卡修正['交易对手账卡号'].replace(df_换卡最后一条.iloc[i-1,1],df_换卡第一条.iloc[i-1,0])
                print(i, end = ' ')
                print('  replace("'+ str(df_换卡最后一条.iloc[i-1,1])+'","'+str(df_换卡第一条.iloc[i-1,0])+'")' )
            elif (   ((df_换卡第一条.iloc[i,8]=='进') and (abs(df_换卡最后一条.iloc[i-1,7] + df_换卡第一条.iloc[i,6] - df_换卡第一条.iloc[i,7]) < 0.0001) ) 
                or ((df_换卡第一条.iloc[i,8]=='出') and (abs(df_换卡最后一条.iloc[i-1,7] - df_换卡第一条.iloc[i,6] - df_换卡第一条.iloc[i,7]) < 0.0001) ) ):
                all_流水卡修正['交易卡号']       =       all_流水卡修正['交易卡号'].replace(df_换卡最后一条.iloc[i-1,0],df_换卡第一条.iloc[i,0])
                all_流水卡修正['交易对手账卡号'] = all_流水卡修正['交易对手账卡号'].replace(df_换卡最后一条.iloc[i-1,0],df_换卡第一条.iloc[i,0])
                print(i, end = ' ')
                print('replace("'+ str(df_换卡最后一条.iloc[i-1,0])+'","'+str(df_换卡第一条.iloc[i,0])+'")' )

            elif all(  (     (df_换卡第一条.iloc[i,8]=='进')
                      & (abs(df_换卡最后一条.iloc[i-1,7] + df_换卡第一条.iloc[i,6] - df_换卡第一条.iloc[i,7]) > 0.0001) 
                      & (abs(df_换卡.iloc[df_换卡第一条.iloc[[i],:].index-1,7] + df_换卡第一条.iloc[i,6] - df_换卡第一条.iloc[i,7]) < 0.0001) 
                     ) 
                 | (     (df_换卡第一条.iloc[i,8]=='出')
                      & (abs(df_换卡最后一条.iloc[i-1,7] - df_换卡第一条.iloc[i,6] - df_换卡第一条.iloc[i,7]) > 0.0001) 
                      & (abs(df_换卡.iloc[df_换卡第一条.iloc[[i],:].index-1,7] - df_换卡第一条.iloc[i,6] - df_换卡第一条.iloc[i,7]) < 0.0001) 
                     )
                 ):
                if all(( (df_换卡.iloc[df_换卡最后一条.iloc[[i-1],:].index+1,8]=='进') &\
                      (abs(df_换卡最后一条.iloc[i-1,7] + df_换卡.iloc[df_换卡最后一条.iloc[[i-1],:].index+1,6] - df_换卡.iloc[df_换卡最后一条.iloc[[i-1],:].index+1,7]) < 0.0001) 
                     ) | \
                    ( (df_换卡.iloc[df_换卡最后一条.iloc[[i-1],:].index+1,8]=='出') & \
                      (abs(df_换卡最后一条.iloc[i-1,7] - df_换卡.iloc[df_换卡最后一条.iloc[[i-1],:].index+1,6] - df_换卡.iloc[df_换卡最后一条.iloc[[i-1],:].index+1,7]) < 0.0001) 
                     )
                   ):
                    if df_换卡最后一条.iloc[i-1,0] != df_换卡第一条.iloc[i,0]:# 换卡中间穿插卡号为空，但是换卡前&空&换卡后余额有连续衔接的
                        all_流水卡修正['交易卡号']       =       all_流水卡修正['交易卡号'].replace(df_换卡最后一条.iloc[i-1,0],df_换卡第一条.iloc[i,0])
                        all_流水卡修正['交易对手账卡号'] = all_流水卡修正['交易对手账卡号'].replace(df_换卡最后一条.iloc[i-1,0],df_换卡第一条.iloc[i,0])
                        print(i, end = ' ')
                        print('replace("'+ str(df_换卡最后一条.iloc[i-1,0])+'","'+str(df_换卡第一条.iloc[i,0])+'")')
                    else:                                                   # 不是换卡，穿插在一张卡中间的另一张卡，但是穿插前后余额有连续衔接的 
                        all_流水卡修正['交易卡号']       =       all_流水卡修正['交易卡号'].replace(df_换卡最后一条.iloc[i-1,0],df_换卡.iloc[df_换卡最后一条.iloc[[i-1],:].index+1,0].iloc[0])
                        all_流水卡修正['交易对手账卡号'] = all_流水卡修正['交易对手账卡号'].replace(df_换卡最后一条.iloc[i-1,0],df_换卡.iloc[df_换卡最后一条.iloc[[i-1],:].index+1,0].iloc[0])
                        print(i, end = ' ')
                        print('replace("'+ str(df_换卡最后一条.iloc[i-1,0])+'","'+str(df_换卡.iloc[df_换卡最后一条.iloc[[i-1],:].index+1,0].iloc[0])+'")')
            else:
                print('否')
                break



# 户名填充
# 方法一：依赖账户开户信息表汇总的all_账户
df_账户 = all_账户[all_账户['账户开户名称'] != '']

#  执行了手动修正卡号则屏蔽
df_流水 = all_流水卡修正  #  df_流水 是手动修正卡号后的流水汇总 ，不需要手动就用账号卡号唯一关系修正的 all_流水卡修正 

dfs = pd.merge(df_流水,df_账户.drop_duplicates(subset='交易卡号'),how = 'left',on = '交易卡号').iloc[:,:(df_流水.shape[1]+2)]
dfs['交易户名'][dfs['账户开户名称'].notnull()] = dfs['账户开户名称'][dfs['账户开户名称'].notnull()]
dfs['交易证件号码'][dfs['开户人证件号码'].notnull()] = dfs['开户人证件号码'][dfs['开户人证件号码'].notnull()]
df = dfs.iloc[:,:34]
df.columns = df_流水.columns
print(df.shape)
# 方法二：依赖交易对手卡号、对手户名的唯一对应关系
df_户名 = df[(df['交易对手账卡号'].isin(df['交易卡号'][df['交易户名']==''])) & (df['对手户名']!='')]  # 对手卡号中的主账号且作为对手卡号出现时对手户名非空，作为本方卡号出现时户名缺失
df_a1 = df_户名.groupby(['交易对手账卡号','对手户名']).size().reset_index()
df_a2 = df_a1.groupby(['交易对手账卡号'],as_index=False).size()
account2 = df_a1[df_a1['交易对手账卡号'].isin(df_a2[df_a2==1].index)]
account2.to_excel(output_path + "卡号户名唯一对应.xlsx",index = None)
print('卡号户名对应Done!')
dfs2 = pd.merge(df,account2,how = 'left',left_on='交易卡号',right_on='交易对手账卡号')
print(dfs2.columns)
dfs2['交易户名'][dfs2['对手户名_y'].notnull()] = dfs2['对手户名_y'][dfs2['对手户名_y'].notnull()]
df = dfs2.iloc[:,:34]
df.columns = df_流水.columns
print(df.shape)
# # 方法三：依赖手工整理的银行卡号、姓名唯一对应表格
# account3 = pd.read_excel('E:\\7.3徐州网赌\\银行卡人员信息整理(提供给公司).xls',sheet_name = '去重',encoding = 'gbk')
# dfs3 = pd.merge(df,account3.drop_duplicates(subset='银行卡号'),how = 'left',left_on='交易卡号',right_on='银行卡号')
# dfs3['交易户名'][dfs3['姓名'].notnull()] = dfs3['姓名'][dfs3['姓名'].notnull()]
# df = dfs3.iloc[:,:34]
# df.columns = df_流水.columns
# print(df.shape)
df_流水户名 = df
df_流水户名.to_csv(output_path +"流水户名.csv",index = None)
print('流水户名Done!')

# #--------------------------------------------------保留7月以后---------------------------------------------------------#
# df = df[(df['交易时间'].str[0:4].isin(['2020']) ) & (df['交易时间'].str[5:7].isin(['07','08','09','10','11']) )]
# #--------------------------------------------------保留7月以后---------------------------------------------------------#

for k in df['交易户名'].drop_duplicates():
    if k !='':
        if not os.path.exists(output_path + k):
            os.mkdir(output_path + k)
        print(output_path + k)

df_S1 = df.groupby('交易卡号').size().reset_index()
df_S2 = pd.DataFrame(columns = ['交易卡号','Size','False','True'])
df_S3 = pd.DataFrame(columns = ['交易卡号','交易户名','证件号码','夜间交易次数','最早时间','最晚时间','余额','总和','净和','入向','出向','入向对手数','出向对手数','收付检验起始值','Path'])
for i in df_S1.index:
    df_every = df.groupby('交易卡号').get_group(df_S1['交易卡号'][i])
    bool_S = df_every.duplicated(subset = ['交易卡号','交易时间','交易金额','交易余额','收付标志','交易对手账卡号','交易币种'],keep = 'last').value_counts()
    df_S2.loc[i] = {'交易卡号':df_every['交易卡号'].iloc[0],
                        'Size':df_every.shape[0],
                        'False':bool_S[0],
                        'True':bool_S.map(lambda x:0 if len(bool_S)==1 else bool_S[1]).values[0]
                    }
    df_every = df_every.drop_duplicates(subset = ['交易卡号','交易时间','交易金额','交易余额','收付标志','交易对手账卡号','交易币种'],keep = 'last')
    if '交易明细信息' in df_every['path'].iloc[0]:
#         户名+卡号+银行
        df_every.to_excel(output_path+df_every['交易户名'].iloc[0]+"\\" + str(df_every['交易户名'].iloc[0])+str(df_every['交易卡号'].iloc[0])+str(df_every['path'].str.split('\\\\').iloc[0][-1]).split('银行')[0]+".xlsx",index = None)
    else:
#         户名+ 卡号
        df_every.to_excel(output_path+df_every['交易户名'].iloc[0]+"\\" + str(df_every['交易户名'].iloc[0])+str(df_every['交易卡号'].iloc[0])+".xlsx",index = None)
    df_night = df_every[df_every['交易时间'].str[11:13].isin(['00','01','02','03','04','05','06'])]
    df_S3.loc[i] = {'交易卡号':df_every['交易卡号'].iloc[0],
                    '交易户名':df_every['交易户名'].iloc[0],
                    '证件号码':df_every['交易证件号码'].iloc[0],
                    '夜间交易次数':len(df_night),
                    '最早时间':df_every['交易时间'].min(),
                    '最晚时间':df_every['交易时间'].max(),
                    '余额':df_every['交易余额'].loc[df_every.index.max()],
                    '总和':df_every['交易金额'].sum(),
                    '净和':round(df_every['交易金额'][df_every['收付标志']=='进'].sum()-df_every['交易金额'][df_every['收付标志']=='出'].sum(), 2),
                    '入向':df_every['交易金额'][df_every['收付标志']=='进'].sum(),
                    '出向':df_every['交易金额'][df_every['收付标志']=='出'].sum(),
                    '入向对手数':len(df_every['交易对手账卡号'][df_every['收付标志']=='进'].drop_duplicates()),
                    '出向对手数':len(df_every['交易对手账卡号'][df_every['收付标志']=='出'].drop_duplicates()),
                    '收付检验起始值':df_every['交易余额'].iloc[0]+df_every.apply(lambda x: x['交易金额'] if x['收付标志']=='出' else -x['交易金额'],axis=1).iloc[0],
                    'Path':'\n'.join(df_every['path'].drop_duplicates().tolist())
                   }
    print(i,end = ' ')
df_S = pd.merge(df_S3,df_S2,how = 'left',on = '交易卡号')

dfs_账户银行 = pd.merge(df_S,df_账户.drop_duplicates(subset='交易卡号'),how = 'left',on = '交易卡号')
df_S['开户网点'] = dfs_账户银行['开户网点']
df_S['账户状态'] = dfs_账户银行['账户状态']
df_S.to_excel(output_path + "交易统计.xlsx",index = None)
# 统计完了，按照六大要素+币种 删除重复项（因为手动改了-101与-102的双向交易的卡号后缀，-101出-102进这类记录会造成自实体链接，不影响）
all_流水完整 = df.drop_duplicates(subset = ['交易卡号','交易时间','交易金额','交易余额','收付标志','交易对手账卡号','交易币种'],keep = 'last')
all_流水完整.to_csv(output_path + "流水完整.csv",index = None)
print(all_流水完整.shape)

手工微调后(4062471, 34)
账号卡号对应Done!
Index([  '交易卡号_x',     '交易账号', '查询反馈结果原因',     '交易户名',   '交易证件号码',     '交易时间',
           '交易金额',     '交易余额',     '收付标志',  '交易对手账卡号',     '现金标志',     '对手户名',
         '对手身份证号',   '对手开户银行',     '摘要说明',     '交易币种',   '交易网点名称',     '交易场所',
          '交易发生地',   '交易是否成功',      '传票号',     'IP地址',    'MAC地址',   '对手交易余额',
          '交易流水号',       '渠道',      '日志号',     '凭证种类',      '凭证号',    '交易柜员号',
             '备注',     '交易类型',     'path',       '序号',   '交易卡号_y',          0],
      dtype='object')


D:\Users\ThinkPad\anaconda3\lib\site-packages\ipykernel_launcher.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(4286258, 34)
                   交易账号                 交易卡号     0
11  0200244101026745836  6212260200126966692  2409
12  0200244101026745836  6212260200195771890     1
1 replace("6212260200195771890","6212260200126966692")
                   交易账号                 交易卡号     0
68  1102130401012326480  6212261102044045659     2
69  1102130401012326480  6222031102003997369  2083
1 replace("6212261102044045659","6222031102003997369")
                   交易账号                 交易卡号     0
74  1103046601003683097  6212251103000262439  3108
75  1103046601003683097  6217231103001889176     4
1 replace("6217231103001889176","6212251103000262439")
               交易账号                 交易卡号     0
82  110901100746730  6228480136729683476    60
83  110901100746730  6228480136756146876  1536
1 replace("6228480136729683476","6228480136756146876")
                交易账号              交易卡号   0
112  120231420500015  5240115712686869  70
113  120231420500015  6214866124686869  65
1 replace("6214866124686869","5240115

1 replace("6214835541788012","6214835543017519")
                交易账号              交易卡号     0
858  591000796400001  4682035910003238    26
859  591000796400001  6214855912558265  2169
1 replace("4682035910003238","6214855912558265")
                 交易账号                 交易卡号    0
1499  626301100231444  6228480388090434779  402
1500  626301100231444  6228480389559342370    1
1 replace("6228480388090434779","6228480389559342370")
                 交易账号                 交易卡号     0
1505  630101101618407  6228482838833466775  1434
1506  630101101618407  6228482838835549172     4
1 replace("6228482838833466775","6228482838835549172")
                 交易账号                 交易卡号      0
1507  630201100611252  6228412834553528873      7
1508  630201100611252  6228412834553530978  10364
1 replace("6228412834553530978","6228412834553528873")
                 交易账号                 交易卡号      0
1509  630201100611955  6228412834555080279     10
1510  630201100611955  6228412834555392179  10658
1 replace("

D:\Users\ThinkPad\anaconda3\lib\site-packages\ipykernel_launcher.py:139: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Users\ThinkPad\anaconda3\lib\site-packages\ipykernel_launcher.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(4286258, 34)
卡号户名对应Done!
Index([     '交易卡号',      '交易账号',  '查询反馈结果原因',      '交易户名',    '交易证件号码',
            '交易时间',      '交易金额',      '交易余额',      '收付标志', '交易对手账卡号_x',
            '现金标志',    '对手户名_x',    '对手身份证号',    '对手开户银行',      '摘要说明',
            '交易币种',    '交易网点名称',      '交易场所',     '交易发生地',    '交易是否成功',
             '传票号',      'IP地址',     'MAC地址',    '对手交易余额',     '交易流水号',
              '渠道',       '日志号',      '凭证种类',       '凭证号',     '交易柜员号',
              '备注',      '交易类型',      'path',        '序号', '交易对手账卡号_y',
          '对手户名_y',           0],
      dtype='object')


D:\Users\ThinkPad\anaconda3\lib\site-packages\ipykernel_launcher.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(4286258, 34)
流水户名Done!
O:\舒城最近清洗\敖日格力玛
O:\舒城最近清洗\洪丽萍
O:\舒城最近清洗\张政文
O:\舒城最近清洗\金芳芳
O:\舒城最近清洗\金燕芳
O:\舒城最近清洗\周辉
O:\舒城最近清洗\吴宇微
O:\舒城最近清洗\胡晶晶
O:\舒城最近清洗\严超达
O:\舒城最近清洗\王绍群
O:\舒城最近清洗\叶莲英
O:\舒城最近清洗\雷斌
O:\舒城最近清洗\王晨杨
O:\舒城最近清洗\潘开成
O:\舒城最近清洗\侍也
O:\舒城最近清洗\骆发友
O:\舒城最近清洗\泮敬委
O:\舒城最近清洗\罗钱
O:\舒城最近清洗\徐志强
O:\舒城最近清洗\辛明光
O:\舒城最近清洗\田忠恩
O:\舒城最近清洗\刘立平
O:\舒城最近清洗\胡淑维
O:\舒城最近清洗\李秀萍
O:\舒城最近清洗\罗海花
O:\舒城最近清洗\周国强
O:\舒城最近清洗\徐忠兴
O:\舒城最近清洗\牛方侠
O:\舒城最近清洗\刘西江
O:\舒城最近清洗\李强强
O:\舒城最近清洗\习端阳
O:\舒城最近清洗\谢应超
O:\舒城最近清洗\方秀云
O:\舒城最近清洗\段昌银
O:\舒城最近清洗\陈宝
O:\舒城最近清洗\程笑
O:\舒城最近清洗\董俊琳
O:\舒城最近清洗\黄俊强
O:\舒城最近清洗\钟武广
O:\舒城最近清洗\冯晓丹
O:\舒城最近清洗\许福英
O:\舒城最近清洗\赖锦锋
O:\舒城最近清洗\傅华丽
O:\舒城最近清洗\阮仕玉
O:\舒城最近清洗\程坤
O:\舒城最近清洗\贾强
O:\舒城最近清洗\曾棉亮
O:\舒城最近清洗\钟伟东
O:\舒城最近清洗\赵航
O:\舒城最近清洗\杨朋林
O:\舒城最近清洗\陈元志
O:\舒城最近清洗\温海鸿
O:\舒城最近清洗\朱雁秋
O:\舒城最近清洗\林斌宇
O:\舒城最近清洗\林小明
O:\舒城最近清洗\林美桂
O:\舒城最近清洗\吴卢伟
O:\舒城最近清洗\李以谢
O:\舒城最近清洗\林徐冬
O:\舒城最近清洗\石双林
O:\舒城最近清洗\蔡志雄
O:\舒城最近清洗\沈维军
O:\舒城最近清洗\刘爽
O:\舒城最近清洗\陈建婷
O:\舒城最近清洗\谈永成
O:\舒城最近清洗\钟立红
O:\舒城最近清洗\钟沅北
O:\舒城最近清洗\唐园创
O:\舒城最近清洗\冯宏标
O:\舒城最近清洗\辛文好
O:\舒

O:\舒城最近清洗\穆仁勇
O:\舒城最近清洗\詹国强
O:\舒城最近清洗\江军秀
O:\舒城最近清洗\陈映君
O:\舒城最近清洗\陈桂斌
O:\舒城最近清洗\杨佳祥
O:\舒城最近清洗\蔡邦书
O:\舒城最近清洗\葛摇兰
O:\舒城最近清洗\姚文飞
O:\舒城最近清洗\孔凤芹
O:\舒城最近清洗\田康
O:\舒城最近清洗\李繁星
O:\舒城最近清洗\王素萍
O:\舒城最近清洗\高猛
O:\舒城最近清洗\郭传才
O:\舒城最近清洗\任旭东
O:\舒城最近清洗\陈裙裙
O:\舒城最近清洗\高中庆
O:\舒城最近清洗\陈英思
O:\舒城最近清洗\王敬智
O:\舒城最近清洗\成雅
O:\舒城最近清洗\王通飞
O:\舒城最近清洗\周锡浩
O:\舒城最近清洗\何静
O:\舒城最近清洗\彭伟萩
O:\舒城最近清洗\朱光明
O:\舒城最近清洗\王旭辉
O:\舒城最近清洗\汪呆红
O:\舒城最近清洗\石朋朋
O:\舒城最近清洗\朱传菊
O:\舒城最近清洗\李兴顺
O:\舒城最近清洗\郭圣宝
O:\舒城最近清洗\季洪松
O:\舒城最近清洗\姚清义
O:\舒城最近清洗\王云龙
O:\舒城最近清洗\李兴博
O:\舒城最近清洗\王延峰
O:\舒城最近清洗\姚嘉琪
O:\舒城最近清洗\梁增铜
O:\舒城最近清洗\方春春
O:\舒城最近清洗\刘群喜
O:\舒城最近清洗\李红民
O:\舒城最近清洗\张英强
O:\舒城最近清洗\柳云龙
O:\舒城最近清洗\肖露
O:\舒城最近清洗\韦英文
O:\舒城最近清洗\蒙凤美
O:\舒城最近清洗\张云侠
O:\舒城最近清洗\杜功勋
O:\舒城最近清洗\苏成香
O:\舒城最近清洗\张旺兵
O:\舒城最近清洗\胡庭华
O:\舒城最近清洗\唐纯勇
O:\舒城最近清洗\顾明杰
O:\舒城最近清洗\王林峰
O:\舒城最近清洗\林太胜
O:\舒城最近清洗\凌俊宇
O:\舒城最近清洗\徐仙山
O:\舒城最近清洗\张爱霞
O:\舒城最近清洗\李成志
O:\舒城最近清洗\许再欢
O:\舒城最近清洗\蔡惠娜
O:\舒城最近清洗\陈彬
O:\舒城最近清洗\王艳妮
O:\舒城最近清洗\庄惠敏
O:\舒城最近清洗\任家林
O:\舒城最近清洗\罗振兴
O:\舒城最近清洗\郭钜章
O:\舒城最近清洗\林可鑫
O:\舒城最近清洗\周东红
O:\舒城最近清洗\刘永彪
O:\舒城最近清洗\赵翠

In [138]:
df_换卡第一条

,交易卡号,交易账号,查询反馈结果原因,交易户名,交易证件号码,交易时间,交易金额,交易余额,收付标志,交易对手账卡号,...,交易流水号,渠道,日志号,凭证种类,凭证号,交易柜员号,备注,交易类型,path,序号
0,6222081505000768444,1505208801023298320,交易成功,,,2020-07-06 08:09:40,5.5,40.11,出,243300133,...,1051,,,,,228,,消费,H:\11.30舒城\二级32个已调账户2020.12.1\3405642320201028...,46623
3509,6222081505000831580,1505208801023298320,交易成功,,,2020-09-04 11:52:09,1.0,1.00,进,6222620660002011745,...,4570,,,,,340,,转账,H:\11.30舒城\二级32个已调账户2020.12.1\3405642320201028...,55066
10508,6212261505002210299,1505208801023298320,交易成功,,,2020-09-24 15:32:15,0.1,1326.87,出,6236682060000382559,...,11577,,,,,59,,跨行汇款,H:\11.30舒城\二级32个已调账户2020.12.1\3405642320201028...,63608


In [13]:
df_换卡最后一条.iloc[2-1,0]

'6212260200126966692'

In [ ]:
df = all_流水完整
box1 = df[df['交易对手账卡号'].isin(df['交易卡号'])]
col_2 = box1.columns.tolist()+['付款方','收款方']
box2 = pd.DataFrame(box1,columns = col_2)
box2['付款方'][box2['收付标志']=='出'] = box2['交易卡号'][box2['收付标志']=='出']
box2['收款方'][box2['收付标志']=='出'] = box2['交易对手账卡号'][box2['收付标志']=='出']
box2['付款方'][box2['收付标志']=='进'] = box2['交易对手账卡号'][box2['收付标志']=='进']
box2['收款方'][box2['收付标志']=='进'] = box2['交易卡号'][box2['收付标志']=='进']
box_group = box2.groupby(['付款方','收款方'])
box_group.size().to_excel(output_path + "box.xlsx")
print(box2.shape,end = ' ')
print(len(box_group.size().reset_index()))

# 前提是调取的数据以日为最小单位，实际调取至今的话“至今”最小单位时分秒，有误差，例如6217003800028006371
box_AB = pd.DataFrame()
df_AB = df
for i in range(0,len(box_group.size().reset_index())):
        box_every = box_group.get_group(box_group.size().index[i])
        box_every['日期'] = box_every['交易时间'].str[0:10]
        if len(box_every.drop_duplicates(['交易卡号']))==2:
            if box_every[box_every['收付标志']=='出']['日期'].min()>box_every[box_every['收付标志']=='进']['日期'].min():
                box_every = box_every[box_every['日期']>=box_every[box_every['收付标志']=='出']['日期'].min()]
            else:
                box_every = box_every[box_every['日期']>=box_every[box_every['收付标志']=='进']['日期'].min()]
            if len(box_every.drop_duplicates(['交易卡号']))==2:
                if box_every[box_every['收付标志']=='出']['日期'].max()<box_every[box_every['收付标志']=='进']['日期'].max():
                    box_every = box_every[box_every['日期']<=box_every[box_every['收付标志']=='出']['日期'].max()]
                else:
                    box_every = box_every[box_every['日期']<=box_every[box_every['收付标志']=='进']['日期'].max()]
                box_AB = box_AB.append(box_every)
                df_AB = df_AB.drop(box_every[box_every['收付标志']=='出'].index)
                print(str(i)+'去重后保留'+str(df_AB.shape[0]),end = ' ')
            else:
                print(str(i)+'无交集',end = ' ')
        else:
                print(i,end = ' ')
df_流水去重后 = df_AB
df_流水去重后.to_csv(output_path + "流水去重后.csv",index = None)
box_AB.sort_values(by = ['付款方','收款方','交易时间','收付标志'],ascending=True).to_excel(output_path + "流水去重匹配.xlsx") 
df_AB[df_AB['交易对手账卡号'].isin(df['交易卡号'])].to_excel(output_path + "去重后主账号之间交易.xlsx")
print(df_AB.shape,end = '其中主账号交易')
print(box_AB.shape)

In [ ]:
            if (df_换卡[df_换卡['交易卡号']==df_换卡['交易账号']].shape[1] != 0) and (i ==j-1):
                continue

In [60]:
df_换卡最后一条.iloc[[i-1],:]

,交易卡号,交易账号,查询反馈结果原因,交易户名,交易证件号码,交易时间,交易金额,交易余额,收付标志,交易对手账卡号,...,交易流水号,渠道,日志号,凭证种类,凭证号,交易柜员号,备注,交易类型,path,序号
10504,6222081505000831580,1505208801023298320,交易成功,,,2020-09-24 14:26:48,3300.0,26.97,出,6222031913000579707,...,11573,,,,,59,,网转,H:\11.30舒城\二级32个已调账户2020.12.1\3405642320201028...,63599


In [11]:
df_换卡.iloc[df_换卡最后一条.iloc[[i-1],:].index+1]

IndexError: positional indexers are out-of-bounds

In [60]:
df_账号 = df[(df['交易卡号']!=df['交易账号']) & (df['交易账号']!='')] #  提取交易卡号、交易账号的对应关系表
df_a1 = df_账号.groupby(['交易账号','交易卡号']).size().reset_index()
df_a2 = df_a1.groupby(['交易账号'],as_index = False).size()
card = df_a1[df_a1['交易账号'].isin(df_a2[df_a2==1].index)]
card.to_excel(output_path + "卡号账号唯一对应.xlsx",index = None)

'C:\\Users\\ThinkPad\\Desktop\\银行流水\\三建置业\\中行\\3404398220200917093046\\中国银行交易明细信息.csv'

In [33]:
12.98+50.0

62.980000000000004

In [1]:
import pandas as pd
import numpy as np
import re
import os
import datetime
output_path = 'O:\\舒城最近清洗\\'

all_流水 = pd.read_csv(output_path + "流水合并剔除冲正.csv",encoding = 'utf-8',dtype = 'object')
all_流水 = all_流水.applymap(lambda x: re.sub('nan','',str(x)))
all_流水['交易金额'] = all_流水['交易金额'].map(lambda x: float(x))  #交易金额指定浮点型
all_流水['交易余额'] = all_流水['交易余额'].map(lambda x: np.NaN if str(x)=='' else x)
all_流水['交易余额'] = all_流水['交易余额'].map(lambda x: float(x))

all_账户 = pd.read_excel(output_path + "账户合并.xlsx",encoding = 'gbk',dtype = 'object')
all_账户 = all_账户.applymap(lambda x: re.sub('nan','',str(x)))


df = all_流水.sort_values(by = ['交易卡号','交易时间','交易金额'],ascending=[True,True,False]).reset_index().iloc[:,1:]  # 流水汇总的all_流水 更新索引

df_账号 = df[(df['交易卡号']!=df['交易账号']) & (df['交易账号']!='')] #  提取交易卡号、交易账号的对应关系表
df_a1 = df_账号.groupby(['交易账号','交易卡号']).size().reset_index()
df_a2 = df_a1.groupby(['交易账号'],as_index = False).size()
card = df_a1[df_a1['交易账号'].isin(df_a2[df_a2==1].index)]
card.to_excel(output_path + "卡号账号唯一对应.xlsx",index = None)

print('账号卡号对应Done!')
dfs0 = pd.merge(df,card,how = 'left',on='交易账号')
print(dfs0.columns)
dfs0['交易卡号_x'][(dfs0['交易卡号_x']!=dfs0['交易卡号_y']) & (dfs0['交易卡号_y'].notnull())]=dfs0['交易卡号_y'][(dfs0['交易卡号_x']!=dfs0['交易卡号_y']) & (dfs0['交易卡号_y'].notnull())]
all_流水卡修正 = dfs0.iloc[:,:34]
all_流水卡修正.columns = all_流水.columns
all_流水卡修正.to_csv(output_path + "流水卡号.csv",index = None)
print(all_流水卡修正.shape)


df_账号 = all_流水卡修正[ (all_流水卡修正['交易账号']!='')] 
df_a1 = df_账号.groupby(['交易账号','交易卡号']).size().reset_index()
df_a2 = df_a1.groupby(['交易账号'],as_index = False).size()
# print(df_a1)
# print(df_a1[df_a1['交易账号'].isin(df_a2[df_a2==2].index)])

for k in range(len(df_a2[df_a2==2])):
    print(df_a1[df_a1['交易账号']==(df_a2[df_a2==2].index[k])])

    df_换卡 = all_流水卡修正.groupby('交易账号').get_group(df_a2[df_a2==2].index[k])
    df_换卡 = df_换卡.sort_values(by = ['交易账号','交易时间','交易金额'],ascending=[True,True,False]).reset_index().iloc[:,1:]
    df_换卡最后一条 = df_换卡.drop_duplicates(subset = ['交易卡号'],keep = 'last')
    df_换卡第一条 = df_换卡.drop_duplicates(subset = ['交易卡号'],keep = 'first')
#     print(df_换卡最后一条.iloc[0])
#     print(df_换卡第一条.iloc[-1,:])
    if df_换卡第一条.iloc[-1,8]=='进':
        if abs(df_换卡最后一条.iloc[0,7] + df_换卡第一条.iloc[-1,6] - df_换卡第一条.iloc[-1,7]) <=0.0001:
            all_流水卡修正['交易卡号']       =       all_流水卡修正['交易卡号'].replace(df_换卡最后一条.iloc[0,0],df_换卡最后一条.iloc[-1,0])
            all_流水卡修正['交易对手账卡号'] = all_流水卡修正['交易对手账卡号'].replace(df_换卡最后一条.iloc[0,0],df_换卡最后一条.iloc[-1,0])
            print('replace("'+ str(df_换卡最后一条.iloc[0,0])+'","'+str(df_换卡最后一条.iloc[-1,0])+'")' )
        else:
            print('否')
    elif df_换卡第一条.iloc[-1,8]=='出':
        if abs(df_换卡最后一条.iloc[0,7] - df_换卡第一条.iloc[-1,6] - df_换卡第一条.iloc[-1,7]) <=0.0001:
            all_流水卡修正['交易卡号']       =       all_流水卡修正['交易卡号'].replace(df_换卡最后一条.iloc[0,0],df_换卡最后一条.iloc[-1,0])
            all_流水卡修正['交易对手账卡号'] = all_流水卡修正['交易对手账卡号'].replace(df_换卡最后一条.iloc[0,0],df_换卡最后一条.iloc[-1,0])
            print('replace("'+ str(df_换卡最后一条.iloc[0,0])+'","'+str(df_换卡最后一条.iloc[-1,0])+'")' )
        else:
            print('否')

for j in range(3,df_a2.max()+1):
    
    for k in range(len(df_a2[df_a2==j])):
        print(df_a1[df_a1['交易账号']==(df_a2[df_a2==j].index[k])])

        df_换卡 = all_流水卡修正.groupby('交易账号').get_group(df_a2[df_a2==j].index[k])
        df_换卡 = df_换卡.sort_values(by = ['交易账号','交易时间','交易金额'],ascending=[True,True,False]).reset_index().iloc[:,1:]
        df_换卡最后一条 = df_换卡.drop_duplicates(subset = ['交易卡号'],keep = 'last')
        df_换卡第一条 = df_换卡.drop_duplicates(subset = ['交易卡号'],keep = 'first')
#     print(df_换卡最后一条.iloc[0])
#     print(df_换卡第一条.iloc[-1,:])
        for i in range(1,j):
            if ((df_换卡第一条.iloc[i,8]=='进') and (abs(df_换卡最后一条.iloc[i-1,7] + df_换卡第一条.iloc[i,6] - df_换卡第一条.iloc[i,7]) <=0.0001) ) or ((df_换卡第一条.iloc[i,8]=='出') and (abs(df_换卡最后一条.iloc[i-1,7] - df_换卡第一条.iloc[i,6] - df_换卡第一条.iloc[i,7]) <=0.0001) ):
                all_流水卡修正['交易卡号']       =       all_流水卡修正['交易卡号'].replace(df_换卡最后一条.iloc[i-1,0],df_换卡最后一条.iloc[i,0])
                all_流水卡修正['交易对手账卡号'] = all_流水卡修正['交易对手账卡号'].replace(df_换卡最后一条.iloc[i-1,0],df_换卡最后一条.iloc[i,0])
                print(i, end = ' ')
                print('replace("'+ str(df_换卡最后一条.iloc[i-1,0])+'","'+str(df_换卡最后一条.iloc[i,0])+'")' )

            elif (df_换卡最后一条.iloc[i-1,0]== df_换卡最后一条.iloc[i-1,1]) and (((df_换卡第一条.iloc[i+1,8]=='进') and (abs(df_换卡最后一条.iloc[i,7] + df_换卡第一条.iloc[i+1,6] - df_换卡第一条.iloc[i+1,7]) <=0.0001) ) or ((df_换卡第一条.iloc[i+1,8]=='出') and (abs(df_换卡最后一条.iloc[i,7] - df_换卡第一条.iloc[i+1,6] - df_换卡第一条.iloc[i+1,7]) <=0.0001) )):
                all_流水卡修正['交易卡号']       =       all_流水卡修正['交易卡号'].replace(df_换卡最后一条.iloc[i-1,1],df_换卡最后一条.iloc[i,0])
                print(i, end = ' ')
                print('replace("'+ str(df_换卡最后一条.iloc[i,1])+'","'+str(df_换卡最后一条.iloc[i,0])+'")' )
            else:
                print('否')
                break


账号卡号对应Done!
Index([  '交易卡号_x',     '交易账号', '查询反馈结果原因',     '交易户名',   '交易证件号码',     '交易时间',
           '交易金额',     '交易余额',     '收付标志',  '交易对手账卡号',     '现金标志',     '对手户名',
         '对手身份证号',   '对手开户银行',     '摘要说明',     '交易币种',   '交易网点名称',     '交易场所',
          '交易发生地',   '交易是否成功',      '传票号',     'IP地址',    'MAC地址',   '对手交易余额',
          '交易流水号',       '渠道',      '日志号',     '凭证种类',      '凭证号',    '交易柜员号',
             '备注',     '交易类型',     'path',       '序号',   '交易卡号_y',          0],
      dtype='object')


D:\Users\ThinkPad\anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(4062471, 34)
                   交易账号                 交易卡号     0
11  0200244101026745836  6212260200126966692  2409
12  0200244101026745836  6212260200195771890     1
否
                   交易账号                 交易卡号     0
68  1102130401012326480  6212261102044045659     2
69  1102130401012326480  6222031102003997369  2083
replace("6212261102044045659","6222031102003997369")
                   交易账号                 交易卡号     0
74  1103046601003683097  6212251103000262439  3108
75  1103046601003683097  6217231103001889176     4
replace("6217231103001889176","6212251103000262439")
               交易账号                 交易卡号     0
82  110901100746730  6228480136729683476    60
83  110901100746730  6228480136756146876  1536
replace("6228480136729683476","6228480136756146876")
                交易账号              交易卡号   0
112  120231420500015  5240115712686869  70
113  120231420500015  6214866124686869  65
否
                    交易账号                 交易卡号      0
125  1207261201104406735  621226120701563

否
                 交易账号                 交易卡号    0
1509  626301100231444  6228480388090434779  402
1510  626301100231444  6228480389559342370    1
replace("6228480388090434779","6228480389559342370")
                 交易账号                 交易卡号     0
1515  630101101618407  6228482838833466775  1434
1516  630101101618407  6228482838835549172     4
replace("6228482838833466775","6228482838835549172")
                 交易账号                 交易卡号      0
1517  630201100611252  6228412834553528873      7
1518  630201100611252  6228412834553530978  10364
replace("6228412834553530978","6228412834553528873")
                 交易账号                 交易卡号      0
1519  630201100611955  6228412834555080279     10
1520  630201100611955  6228412834555392179  10658
replace("6228412834555392179","6228412834555080279")
                 交易账号                 交易卡号     0
1529  635101103575238  6228480389555382479  3477
1530  635101103575238  6228480389558414071    72
replace("6228480389555382479","62284803895584140

IndexError: single positional indexer is out-of-bounds

In [84]:
# # 卡号修正
# # 汇总时已经填充了交易账号，如果交易卡号、交易账号是唯一对应关系，实际应该依赖交易账号填充对应的交易卡号
# df = all_流水.sort_values(by = ['交易卡号','交易时间','交易金额'],ascending=[True,True,False]).reset_index().iloc[:,1:]  # 流水汇总的all_流水 更新索引

# # 手工微调
# df['交易卡号'] = df['交易卡号'].str.split('_', 1,expand = True).iloc[:,0] 
# df['交易卡号'] = df['交易卡号'].str.split('-', 1,expand = True).iloc[:,0]

# # df = df[~((df['交易对手账卡号']=='6236681460012372435') & (df['对手户名']=='吴晶晶'))]
# # df = df[~((df['交易对手账卡号']=='6230780100017400236') & (df['对手户名']=='吴晶晶'))]
# # df = df[~((df['交易对手账卡号']=='6228411070343158413') & (df['对手户名']=='毛霓姣'))]
# # df = df[~((df['交易对手账卡号']=='6212261207008544728') & (df['对手户名']=='唐文强'))]

# # df['对手户名'] = df['对手户名'].replace('邓?婷','邓雲婷')

# df['交易对手账卡号']= df['交易对手账卡号'].replace('21*****90','215500690')
# df['交易对手账卡号']= df['交易对手账卡号'].replace('24*****33','243300133')
# df['交易对手账卡号']= df['交易对手账卡号'].replace('21*****93','215500693')
# df['交易对手账卡号']= df['交易对手账卡号'].replace('22*****12','220800312')


# # df['交易户名'][df['交易卡号']=='6230361104131659666'] = df['交易户名'][df['交易卡号']=='6230361104131659666'].replace('','范金锋')

# print('手工微调后',end = '')
# print(df.shape)



# def groupa1(df_g):
#     return df_g.groupby(['交易账号','交易卡号'],as_index = False).size()
# def groupa2(df_g):
#     return df_g.groupby(['交易卡号'],as_index = False).size()


# df_账号 = df[(df['交易卡号']!=df['交易账号']) & (df['交易账号']!='')] #  提取交易卡号、交易账号的对应关系表
# df_a1 = df_账号.groupby(['交易账号','交易卡号']).size().reset_index()
# df_a2 = df_a1.groupby(['交易账号'],as_index = False).size()
# card = df_a1[df_a1['交易账号'].isin(df_a2[df_a2==1].index)]


# df_开户a1 = groupa1( df ).reset_index()
# df_开户a2 = groupa2(df_开户a1)
# card_明细仅账号 = df_开户a1[ ( df_开户a1['交易账号'].isin(df_开户a2[df_开户a2==1].index) )&( df_开户a1['交易账号'] == df_开户a1['交易卡号'] )]
# card_开户对应卡号 = groupa1( all_账户[all_账户['交易账号'].isin( card_明细仅账号['交易账号'] )] ).reset_index()
# card = card.append(card_开户对应卡号)

# card.to_excel(output_path + "卡号账号唯一对应.xlsx",index = None)

# print('账号卡号对应Done!')
# dfs0 = pd.merge(df,card,how = 'left',on='交易账号')
# print(dfs0.columns)
# dfs0['交易卡号_x'][(dfs0['交易卡号_x']!=dfs0['交易卡号_y']) & (dfs0['交易卡号_y'].notnull())]=dfs0['交易卡号_y'][(dfs0['交易卡号_x']!=dfs0['交易卡号_y']) & (dfs0['交易卡号_y'].notnull())]
# all_流水卡修正 = dfs0.iloc[:,:34]
# all_流水卡修正.columns = all_流水.columns
# # all_流水卡修正.to_csv(output_path + "流水卡号.csv",index = None)
# print(all_流水卡修正.shape)


# df_换卡 = all_流水卡修正.groupby('交易账号').get_group(df_a2[df_a2==2].index[0])
# df_换卡 = df_换卡.sort_values(by = ['交易账号','交易时间','交易金额'],ascending=[True,True,False]).reset_index().iloc[:,1:]
df_换卡最后一条 = df_换卡[df_换卡['交易卡号']!=df_换卡['交易账号']].drop_duplicates(subset = ['交易卡号'],keep = 'last')
df_换卡第一条 = df_换卡[df_换卡['交易卡号']!=df_换卡['交易账号']].drop_duplicates(subset = ['交易卡号'],keep = 'first')
print(df_换卡最后一条.iloc[[0]])
print(df_换卡第一条.iloc[[-1],:])
for i in range(1,2):
    if i == len(df_换卡第一条):
        all_流水卡修正['交易卡号']       =       all_流水卡修正['交易卡号'].replace(df_换卡最后一条.iloc[i-1,1],df_换卡第一条.iloc[i-1,0])
        all_流水卡修正['交易对手账卡号'] = all_流水卡修正['交易对手账卡号'].replace(df_换卡最后一条.iloc[i-1,1],df_换卡第一条.iloc[i-1,0])
        print(i, end = ' ')
        print('  replace("'+ str(df_换卡最后一条.iloc[i-1,1])+'","'+str(df_换卡第一条.iloc[i-1,0])+'")' )
    elif (   ((df_换卡第一条.iloc[i,8]=='进') and (abs(df_换卡最后一条.iloc[i-1,7] + df_换卡第一条.iloc[i,6] - df_换卡第一条.iloc[i,7]) < 0.0001) ) 
        or ((df_换卡第一条.iloc[i,8]=='出') and (abs(df_换卡最后一条.iloc[i-1,7] - df_换卡第一条.iloc[i,6] - df_换卡第一条.iloc[i,7]) < 0.0001) ) ):
        all_流水卡修正['交易卡号']       =       all_流水卡修正['交易卡号'].replace(df_换卡最后一条.iloc[i-1,0],df_换卡第一条.iloc[i,0])
        all_流水卡修正['交易对手账卡号'] = all_流水卡修正['交易对手账卡号'].replace(df_换卡最后一条.iloc[i-1,0],df_换卡第一条.iloc[i,0])
        print(i, end = ' ')
        print('replace("'+ str(df_换卡最后一条.iloc[i-1,0])+'","'+str(df_换卡第一条.iloc[i,0])+'")' )

    elif all(  (     (df_换卡第一条.iloc[i,8]=='进')
                      & (abs(df_换卡最后一条.iloc[i-1,7] + df_换卡第一条.iloc[i,6] - df_换卡第一条.iloc[i,7]) > 0.0001) 
                      & (abs(df_换卡.iloc[df_换卡第一条.iloc[[i],:].index-1,7] + df_换卡第一条.iloc[i,6] - df_换卡第一条.iloc[i,7]) < 0.0001) 
                     ) 
                 | (     (df_换卡第一条.iloc[i,8]=='出')
                      & (abs(df_换卡最后一条.iloc[i-1,7] - df_换卡第一条.iloc[i,6] - df_换卡第一条.iloc[i,7]) > 0.0001) 
                      & (abs(df_换卡.iloc[df_换卡第一条.iloc[[i],:].index-1,7] - df_换卡第一条.iloc[i,6] - df_换卡第一条.iloc[i,7]) < 0.0001) 
                     )
                 ):
        if all(( (df_换卡.iloc[df_换卡最后一条.iloc[[i-1],:].index+1,8]=='进') &\
                      (abs(df_换卡最后一条.iloc[i-1,7] + df_换卡.iloc[df_换卡最后一条.iloc[[i-1],:].index+1,6] - df_换卡.iloc[df_换卡最后一条.iloc[[i-1],:].index+1,7]) < 0.0001) 
                     ) | \
                    ( (df_换卡.iloc[df_换卡最后一条.iloc[[i-1],:].index+1,8]=='出') & \
                      (abs(df_换卡最后一条.iloc[i-1,7] - df_换卡.iloc[df_换卡最后一条.iloc[[i-1],:].index+1,6] - df_换卡.iloc[df_换卡最后一条.iloc[[i-1],:].index+1,7]) < 0.0001) 
                     )
                   ):
            if df_换卡最后一条.iloc[i-1,0] != df_换卡第一条.iloc[i,0]:# 换卡中间穿插卡号为空，但是换卡前&空&换卡后余额有连续衔接的
                all_流水卡修正['交易卡号']       =       all_流水卡修正['交易卡号'].replace(df_换卡最后一条.iloc[i-1,0],df_换卡第一条.iloc[i,0])
                all_流水卡修正['交易对手账卡号'] = all_流水卡修正['交易对手账卡号'].replace(df_换卡最后一条.iloc[i-1,0],df_换卡第一条.iloc[i,0])
                print(i, end = ' ')
                print('replace("'+ str(df_换卡最后一条.iloc[i-1,0])+'","'+str(df_换卡第一条.iloc[i,0])+'")')
            else:                                                   # 不是换卡，穿插在一张卡中间的另一张卡，但是穿插前后余额有连续衔接的 
                all_流水卡修正['交易卡号']       =       all_流水卡修正['交易卡号'].replace(df_换卡最后一条.iloc[i-1,0],df_换卡.iloc[df_换卡最后一条.iloc[[i-1],:].index+1,0].iloc[0])
                all_流水卡修正['交易对手账卡号'] = all_流水卡修正['交易对手账卡号'].replace(df_换卡最后一条.iloc[i-1,0],df_换卡.iloc[df_换卡最后一条.iloc[[i-1],:].index+1,0].iloc[0])
                print(i, end = ' ')
                print('replace("'+ str(df_换卡最后一条.iloc[i-1,0])+'","'+str(df_换卡.iloc[df_换卡最后一条.iloc[[i-1],:].index+1,0].iloc[0])+'")')

    else:
        print('否')
        break


print(df_换卡最后一条)
print(df_换卡第一条)

                    交易卡号                 交易账号 查询反馈结果原因 交易户名 交易证件号码  \
828  6212260200195771890  0200244101026745836     交易成功               

                    交易时间  交易金额     交易余额 收付标志 交易对手账卡号  ...        交易流水号 渠道 日志号  \
828  2020-08-25 08:03:01  14.0  7200.34    出          ...  00000001641          

    凭证种类 凭证号  交易柜员号 备注  交易类型  \
828           00010     自助消费   

                                                  path    序号  
828  H:\10.2舒城\银行资料\3405642320200930104044\中国工商银行交易...  6650  

[1 rows x 34 columns]
                    交易卡号                 交易账号 查询反馈结果原因 交易户名 交易证件号码  \
828  6212260200195771890  0200244101026745836     交易成功               

                    交易时间  交易金额     交易余额 收付标志 交易对手账卡号  ...        交易流水号 渠道 日志号  \
828  2020-08-25 08:03:01  14.0  7200.34    出          ...  00000001641          

    凭证种类 凭证号  交易柜员号 备注  交易类型  \
828           00010     自助消费   

                                                  path    序号  
828  H:\10.2舒城\银行资料\3405642320200930104044\中国工商银行交

In [81]:
df_换卡

,交易卡号,交易账号,查询反馈结果原因,交易户名,交易证件号码,交易时间,交易金额,交易余额,收付标志,交易对手账卡号,...,交易流水号,渠道,日志号,凭证种类,凭证号,交易柜员号,备注,交易类型,path,序号
0,6212260200126966692,0200244101026745836,交易成功,,,2020-07-01 13:41:11,190000.00,190000.91,进,6228430899610557676,...,00000000455,,,,,00337,,他行汇入,H:\10.2舒城\银行资料\3405642320200930104044\中国工商银行交易...,5879
1,6212260200126966692,0200244101026745836,交易成功,,,2020-07-01 14:08:58,500.00,189500.91,出,6228480086821490272,...,00000000456,,,,,00059,,跨行汇款,H:\10.2舒城\银行资料\3405642320200930104044\中国工商银行交易...,5882
2,6212260200126966692,0200244101026745836,交易成功,,,2020-07-01 17:30:13,9500.00,180000.91,出,6228270011237060471,...,00000000457,,,,,00059,,跨行汇款,H:\10.2舒城\银行资料\3405642320200930104044\中国工商银行交易...,5886
3,6212260200126966692,0200244101026745836,交易成功,,,2020-07-02 11:52:05,100.00,179900.91,出,,...,00000000458,,,,,31108,,卡取,H:\10.2舒城\银行资料\3405642320200930104044\中国工商银行交易...,5889
4,6212260200126966692,0200244101026745836,交易成功,,,2020-07-02 12:21:29,400.00,179500.91,出,6228270011237060471,...,00000000459,,,,,00059,,跨行汇款,H:\10.2舒城\银行资料\3405642320200930104044\中国工商银行交易...,5892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2405,6212260200126966692,0200244101026745836,交易成功,,,2020-09-28 22:03:09,500.00,1551.63,出,243300133,...,00000003908,,,,,00235,,消费,H:\10.2舒城\银行资料\3405642320200930104044\中国工商银行交易...,11502
2406,6212260200126966692,0200244101026745836,交易成功,,,2020-09-28 22:03:53,136.00,1415.63,出,243300133,...,00000003909,,,,,00233,,消费,H:\10.2舒城\银行资料\3405642320200930104044\中国工商银行交易...,11524
2407,6212260200126966692,0200244101026745836,交易成功,,,2020-09-29 12:24:00,8.38,1407.25,出,215500690,...,00000003910,,,,,00240,,消费,H:\10.2舒城\银行资料\3405642320200930104044\中国工商银行交易...,11535
2408,6212260200126966692,0200244101026745836,交易成功,,,2020-09-29 13:25:44,500.00,907.25,出,243300133,...,00000003911,,,,,00235,,消费,H:\10.2舒城\银行资料\3405642320200930104044\中国工商银行交易...,11551


In [75]:
df_换卡最后一条

,交易卡号,交易账号,查询反馈结果原因,交易户名,交易证件号码,交易时间,交易金额,交易余额,收付标志,交易对手账卡号,...,交易流水号,渠道,日志号,凭证种类,凭证号,交易柜员号,备注,交易类型,path,序号
2409,6212260200195771890,0200244101026745836,交易成功,,,2020-09-29 14:16:17,150.0,1057.25,进,243300133,...,00000003912,,,,,00235,,微信零钱提现,H:\10.2舒城\银行资料\3405642320200930104044\中国工商银行交易...,11571


In [ ]:
df_换卡.iloc[df_换卡最后一条.iloc[[i-1],:].index+1,0].iloc[0]

In [71]:
df_换卡

,交易卡号,交易账号,查询反馈结果原因,交易户名,交易证件号码,交易时间,交易金额,交易余额,收付标志,交易对手账卡号,...,交易流水号,渠道,日志号,凭证种类,凭证号,交易柜员号,备注,交易类型,path,序号
0,6212260200195771890,0200244101026745836,交易成功,,,2020-07-01 13:41:11,190000.00,190000.91,进,6228430899610557676,...,00000000455,,,,,00337,,他行汇入,H:\10.2舒城\银行资料\3405642320200930104044\中国工商银行交易...,5879
1,6212260200195771890,0200244101026745836,交易成功,,,2020-07-01 14:08:58,500.00,189500.91,出,6228480086821490272,...,00000000456,,,,,00059,,跨行汇款,H:\10.2舒城\银行资料\3405642320200930104044\中国工商银行交易...,5882
2,6212260200195771890,0200244101026745836,交易成功,,,2020-07-01 17:30:13,9500.00,180000.91,出,6228270011237060471,...,00000000457,,,,,00059,,跨行汇款,H:\10.2舒城\银行资料\3405642320200930104044\中国工商银行交易...,5886
3,6212260200195771890,0200244101026745836,交易成功,,,2020-07-02 11:52:05,100.00,179900.91,出,,...,00000000458,,,,,31108,,卡取,H:\10.2舒城\银行资料\3405642320200930104044\中国工商银行交易...,5889
4,6212260200195771890,0200244101026745836,交易成功,,,2020-07-02 12:21:29,400.00,179500.91,出,6228270011237060471,...,00000000459,,,,,00059,,跨行汇款,H:\10.2舒城\银行资料\3405642320200930104044\中国工商银行交易...,5892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2405,6212260200195771890,0200244101026745836,交易成功,,,2020-09-28 22:03:09,500.00,1551.63,出,243300133,...,00000003908,,,,,00235,,消费,H:\10.2舒城\银行资料\3405642320200930104044\中国工商银行交易...,11502
2406,6212260200195771890,0200244101026745836,交易成功,,,2020-09-28 22:03:53,136.00,1415.63,出,243300133,...,00000003909,,,,,00233,,消费,H:\10.2舒城\银行资料\3405642320200930104044\中国工商银行交易...,11524
2407,6212260200195771890,0200244101026745836,交易成功,,,2020-09-29 12:24:00,8.38,1407.25,出,215500690,...,00000003910,,,,,00240,,消费,H:\10.2舒城\银行资料\3405642320200930104044\中国工商银行交易...,11535
2408,6212260200195771890,0200244101026745836,交易成功,,,2020-09-29 13:25:44,500.00,907.25,出,243300133,...,00000003911,,,,,00235,,消费,H:\10.2舒城\银行资料\3405642320200930104044\中国工商银行交易...,11551


In [19]:
i

1

In [8]:
# i = 1
# df_换卡最后一条.iloc[(i-1),7]
df_换卡最后一条.iloc[0]

交易卡号                                      6212260200195771890
交易账号                                      0200244101026745836
查询反馈结果原因                                                 交易成功
交易户名                                                         
交易证件号码                                                       
交易时间                                      2020-08-25 08:03:01
交易金额                                                       14
交易余额                                                  7200.34
收付标志                                                        出
交易对手账卡号                                                      
现金标志                                                       00
对手户名                                                         
对手身份证号                                                       
对手开户银行                                                       
摘要说明                                                       正常
交易币种                                                      CNY
交易网点名称  

In [49]:
2

3

In [53]:
df_换卡第一条

,交易卡号,交易账号,查询反馈结果原因,交易户名,交易证件号码,交易时间,交易金额,交易余额,收付标志,交易对手账卡号,...,交易流水号,渠道,日志号,凭证种类,凭证号,交易柜员号,备注,交易类型,path,序号
0,6228480389555382479,635101103575238,,,,2020-07-18 10:17:57,10.0,10.00,进,243300133,...,,,217967087,,,,代付,EPAY,H:\11.30舒城\二级32个已调账户2020.12.1\3405642320200916...,0
1159,6228480389558414071,635101103575238,,,,2020-08-26 13:38:16,10.0,47.14,出,,...,,,428296596,,,,现支,TERM,H:\11.30舒城\二级32个已调账户2020.12.1\3405642320200916...,1159


In [46]:
df_账号 = all_流水卡修正[ (all_流水卡修正['交易账号']!='')] 
df_a1 = df_账号.groupby(['交易账号','交易卡号']).size().reset_index()
df_a2 = df_a1.groupby(['交易账号'],as_index = False).size()
print(df_a2)
for i in range(3,df_a2.max()+1):
    print(i)
df_a2[df_a2==df_a2.max()]

交易账号
0002991000594642552     1
0002991000595789451     1
0002991000595789477     1
0002991000598238878     1
0002998000036292373     1
                       ..
869101100612759         1
900601100587555         1
95250066601400055069    1
95260066601300158071    1
9550895576272600118     1
Length: 450, dtype: int64
3
4


交易账号
1505208801023298320    4
dtype: int64

In [5]:

df = all_流水完整
box1 = df[df['交易对手账卡号'].isin(df['交易卡号'])]
col_2 = box1.columns.tolist()+['付款方','收款方']
box2 = pd.DataFrame(box1,columns = col_2)
box2['付款方'][box2['收付标志']=='出'] = box2['交易卡号'][box2['收付标志']=='出']
box2['收款方'][box2['收付标志']=='出'] = box2['交易对手账卡号'][box2['收付标志']=='出']
box2['付款方'][box2['收付标志']=='进'] = box2['交易对手账卡号'][box2['收付标志']=='进']
box2['收款方'][box2['收付标志']=='进'] = box2['交易卡号'][box2['收付标志']=='进']
box_group = box2.groupby(['付款方','收款方'])
box_group.size().to_excel(output_path + "box.xlsx")
print(box2.shape,end = ' ')
print(len(box_group.size().reset_index()))

# 前提是调取的数据以日为最小单位，实际调取至今的话“至今”最小单位时分秒，有误差，例如6217003800028006371
box_AB = pd.DataFrame()
df_AB = df
for i in range(0,len(box_group.size().reset_index())):
        box_every = box_group.get_group(box_group.size().index[i])
        box_every['日期'] = box_every['交易时间'].str[0:10]
        if len(box_every.drop_duplicates(['交易卡号']))==2:
            if box_every[box_every['收付标志']=='出']['日期'].min()>box_every[box_every['收付标志']=='进']['日期'].min():
                box_every = box_every[box_every['日期']>=box_every[box_every['收付标志']=='出']['日期'].min()]
            else:
                box_every = box_every[box_every['日期']>=box_every[box_every['收付标志']=='进']['日期'].min()]
            if len(box_every.drop_duplicates(['交易卡号']))==2:
                if box_every[box_every['收付标志']=='出']['日期'].max()<box_every[box_every['收付标志']=='进']['日期'].max():
                    box_every = box_every[box_every['日期']<=box_every[box_every['收付标志']=='出']['日期'].max()]
                else:
                    box_every = box_every[box_every['日期']<=box_every[box_every['收付标志']=='进']['日期'].max()]
                box_AB = box_AB.append(box_every)
                df_AB = df_AB.drop(box_every[box_every['收付标志']=='出'].index)
                print(str(i)+'去重后保留'+str(df_AB.shape[0]),end = ' ')
            else:
                print(str(i)+'无交集',end = ' ')
        else:
                print(i,end = ' ')
df_流水去重后 = df_AB
df_流水去重后.to_csv(output_path + "流水去重后.csv",index = None)
box_AB.sort_values(by = ['付款方','收款方','交易时间','收付标志'],ascending=True).to_excel(output_path + "流水去重匹配.xlsx") 
df_AB[df_AB['交易对手账卡号'].isin(df['交易卡号'])].to_excel(output_path + "去重后主账号之间交易.xlsx")
print(df_AB.shape,end = '其中主账号交易')
print(box_AB.shape)

D:\Users\ThinkPad\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\Users\ThinkPad\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\Users\ThinkPad\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
D:\Users\ThinkPad\anaconda3\lib\site-packages\ipykernel_launcher.py:8: S

(311508, 36) 11322


D:\Users\ThinkPad\anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0去重后保留2287036 1去重后保留2287034 2去重后保留2287000 3去重后保留2286990 4去重后保留2286987 5 6去重后保留2286986 7去重后保留2286983 8去重后保留2286981 9 10去重后保留2286976 11去重后保留2286974 12去重后保留2286972 13去重后保留2286970 14 15去重后保留2286969 16 17去重后保留2286966 18 19 20 21无交集 22 23 24 25 26 27去重后保留2286962 28去重后保留2286958 29去重后保留2286951 30去重后保留2286933 31去重后保留2286932 32去重后保留2286931 33去重后保留2286930 34去重后保留2286925 35去重后保留2286924 36去重后保留2286922 37去重后保留2286899 38去重后保留2286892 39去重后保留2286883 40去重后保留2286881 41去重后保留2286877 42去重后保留2286876 43去重后保留2286874 44去重后保留2286872 45去重后保留2286870 46去重后保留2286869 47去重后保留2286868 48去重后保留2286859 49去重后保留2286852 50去重后保留2286849 51去重后保留2286839 52去重后保留2286826 53去重后保留2286807 54去重后保留2286803 55去重后保留2286775 56去重后保留2286760 57去重后保留2286666 58去重后保留2286665 59去重后保留2286654 60 61去重后保留2286650 62去重后保留2286638 63去重后保留2286625 64去重后保留2286599 65 66 67 68去重后保留2286597 69 70 71去重后保留2286538 72去重后保留2286534 73去重后保留2286441 74去重后保留2286432 75去重后保留2286416 76去重后保留2286415 77去重后保留2286407 78去重后保留2286406 79去重后保留2286405 80去重后保留2286404 81去重后保留2286403 82去重后

569去重后保留2280590 570去重后保留2280581 571 572去重后保留2280573 573去重后保留2280570 574去重后保留2280568 575去重后保留2280561 576去重后保留2280542 577去重后保留2280540 578去重后保留2280539 579去重后保留2280520 580去重后保留2280516 581去重后保留2280509 582去重后保留2280506 583去重后保留2280500 584去重后保留2280489 585去重后保留2280486 586去重后保留2280484 587去重后保留2280460 588去重后保留2280457 589去重后保留2280449 590去重后保留2280447 591去重后保留2280445 592去重后保留2280440 593去重后保留2280436 594去重后保留2280430 595去重后保留2280414 596去重后保留2280412 597去重后保留2280404 598去重后保留2280401 599去重后保留2280400 600去重后保留2280385 601去重后保留2280381 602去重后保留2280372 603去重后保留2280366 604去重后保留2280344 605去重后保留2280337 606去重后保留2280334 607去重后保留2280326 608去重后保留2280318 609去重后保留2280300 610去重后保留2280118 611去重后保留2280113 612去重后保留2280112 613去重后保留2280058 614去重后保留2279929 615去重后保留2279866 616去重后保留2279865 617去重后保留2279861 618去重后保留2279757 619 620去重后保留2279716 621去重后保留2279711 622 623去重后保留2279706 624去重后保留2279700 625去重后保留2279620 626去重后保留2279609 627去重后保留2279606 628无交集 629去重后保留2279605 630 631去重后保留2279522 632去重后保留2279521 633去重后保留2279451 634去重后保留2279424 6

...

9347去重后保留2163229 9348去重后保留2163227 9349去重后保留2163223 9350去重后保留2163222 9351去重后保留2163221 9352去重后保留2163219 9353去重后保留2163218 9354去重后保留2163216 9355去重后保留2163215 9356去重后保留2163214 9357去重后保留2163213 9358去重后保留2163212 9359去重后保留2163211 9360去重后保留2163210 9361去重后保留2163209 9362去重后保留2163208 9363去重后保留2163205 9364去重后保留2163200 9365去重后保留2163199 9366去重后保留2163197 9367去重后保留2163196 9368去重后保留2163195 9369 9370去重后保留2163194 9371去重后保留2163193 9372去重后保留2163192 9373去重后保留2163189 9374去重后保留2163186 9375去重后保留2163185 9376去重后保留2163184 9377去重后保留2163183 9378去重后保留2163181 9379去重后保留2163179 9380去重后保留2163177 9381去重后保留2163176 9382去重后保留2163175 9383去重后保留2163174 9384去重后保留2163173 9385去重后保留2163172 9386去重后保留2163171 9387去重后保留2163170 9388去重后保留2163169 9389去重后保留2163168 9390去重后保留2163167 9391去重后保留2163165 9392去重后保留2163164 9393去重后保留2163163 9394去重后保留2163162 9395去重后保留2163161 9396去重后保留2163160 9397去重后保留2163159 9398去重后保留2163158 9399去重后保留2163156 9400去重后保留2163154 9401去重后保留2163153 9402去重后保留2163152 9403去重后保留2163149 9404去重后保留2163146 9405去重后保留2163145 9406去重后保留

9850去重后保留2158021 9851去重后保留2158020 9852去重后保留2158019 9853去重后保留2158018 9854去重后保留2158016 9855去重后保留2157992 9856去重后保留2157991 9857去重后保留2157988 9858去重后保留2157987 9859去重后保留2157984 9860去重后保留2157983 9861去重后保留2157980 9862去重后保留2157979 9863去重后保留2157978 9864去重后保留2157853 9865去重后保留2157845 9866去重后保留2157844 9867去重后保留2157840 9868去重后保留2157839 9869去重后保留2157837 9870去重后保留2157836 9871去重后保留2157812 9872去重后保留2157682 9873去重后保留2157663 9874去重后保留2157529 9875去重后保留2157528 9876去重后保留2157526 9877去重后保留2157518 9878去重后保留2157516 9879去重后保留2157514 9880去重后保留2157509 9881去重后保留2157503 9882去重后保留2157478 9883去重后保留2157476 9884去重后保留2157471 9885去重后保留2157465 9886去重后保留2157461 9887去重后保留2157437 9888去重后保留2157433 9889去重后保留2157430 9890去重后保留2157428 9891去重后保留2157427 9892去重后保留2157386 9893去重后保留2157320 9894去重后保留2157318 9895去重后保留2157317 9896去重后保留2157214 9897去重后保留2157204 9898去重后保留2157198 9899去重后保留2157196 9900去重后保留2157192 9901去重后保留2157191 9902去重后保留2157188 9903去重后保留2157187 9904去重后保留2157186 9905去重后保留2157185 9906去重后保留2157182 9907去重后保留2157181 9908去重后保留21571

10349去重后保留2151713 10350去重后保留2151706 10351 10352去重后保留2151701 10353去重后保留2151691 10354去重后保留2151558 10355 10356去重后保留2151544 10357无交集 10358 10359去重后保留2151541 10360 10361去重后保留2151533 10362 10363去重后保留2151523 10364去重后保留2151520 10365去重后保留2151507 10366去重后保留2151503 10367 10368去重后保留2151490 10369 10370 10371 10372 10373 10374去重后保留2151488 10375去重后保留2151486 10376 10377 10378去重后保留2151481 10379 10380去重后保留2151471 10381去重后保留2151466 10382去重后保留2151396 10383去重后保留2151219 10384去重后保留2151187 10385去重后保留2151181 10386去重后保留2151177 10387去重后保留2151175 10388去重后保留2151161 10389去重后保留2151150 10390去重后保留2151143 10391 10392 10393去重后保留2151142 10394 10395去重后保留2151119 10396去重后保留2151117 10397去重后保留2151116 10398去重后保留2151115 10399去重后保留2151114 10400去重后保留2151113 10401去重后保留2151112 10402 10403去重后保留2151111 10404去重后保留2151082 10405去重后保留2150938 10406去重后保留2150925 10407去重后保留2150862 10408去重后保留2150760 10409去重后保留2150591 10410去重后保留2150465 10411去重后保留2150427 10412去重后保留2150396 10413去重后保留2150353 10414去重后保留2150318 10415去重后保留2150297 10416去重后保留2150260 1

10835去重后保留2143281 10836去重后保留2143280 10837去重后保留2143277 10838去重后保留2143275 10839去重后保留2143273 10840去重后保留2143272 10841去重后保留2143271 10842去重后保留2143270 10843去重后保留2143269 10844 10845去重后保留2143268 10846去重后保留2143266 10847去重后保留2143265 10848去重后保留2143264 10849去重后保留2143263 10850去重后保留2143261 10851去重后保留2143260 10852去重后保留2143259 10853去重后保留2143254 10854去重后保留2143253 10855去重后保留2143252 10856去重后保留2143251 10857去重后保留2143250 10858去重后保留2143249 10859去重后保留2143248 10860去重后保留2143245 10861去重后保留2143244 10862去重后保留2143239 10863去重后保留2143236 10864去重后保留2143235 10865去重后保留2143234 10866去重后保留2143233 10867去重后保留2143231 10868去重后保留2143228 10869去重后保留2143227 10870去重后保留2143226 10871去重后保留2143224 10872去重后保留2143221 10873去重后保留2143219 10874去重后保留2143213 10875去重后保留2143210 10876去重后保留2143209 10877去重后保留2143208 10878去重后保留2143207 10879去重后保留2143191 10880去重后保留2143190 10881去重后保留2143189 10882去重后保留2143187 10883去重后保留2143185 10884去重后保留2143182 10885去重后保留2143181 10886去重后保留2143179 10887去重后保留2143177 10888去重后保留2143169 10889去重后保留2143167 10890去重后保留2143166 1089

11309去重后保留2136276 11310去重后保留2136274 11311去重后保留2136273 11312去重后保留2136271 11313去重后保留2136270 11314去重后保留2136249 11315去重后保留2136246 11316去重后保留2136213 11317去重后保留2136211 11318去重后保留2136210 11319去重后保留2136182 11320去重后保留2136171 11321去重后保留2136168 (2136168, 34)其中主账号交易(293851, 37)


9月7日接收前已调数据的分批分析

集中转入分散转出账号提取，集中转入整数值占比，分散转出非整数<2000数值占比

----------------------------------------------------拆单卡单表----------------------------------------------------

In [176]:
import pandas as pd
import numpy as np
import re
output_path = 'H:\\11.2舒城\\'

df = pd.read_csv(output_path + "流水完整.csv",encoding = 'utf-8',dtype = object)
df = df.applymap(lambda x: re.sub('nan','',str(x)))
df['交易金额'] = df['交易金额'].map(lambda x: float(x))  #交易金额指定浮点型
df['交易余额'] = df['交易余额'].map(lambda x: np.NaN if str(x)=='' else x)
df['交易余额'] = df['交易余额'].map(lambda x: float(x))
df_备份 = df

df_S1 = df.groupby('交易卡号').size().reset_index()
for i in df_S1.index:
    df_every = df.groupby('交易卡号').get_group(df_S1['交易卡号'][i])

    df_every.to_excel(output_path+"资金流水\\" + str(df_every['交易户名'].iloc[0])+str(df_every['交易卡号'].iloc[0])+".xlsx",index = None)
    print(i,end = ' ')


0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

----------------------------------------------------拆单卡单表----------------------------------------------------